In [2]:


!pip install rasterio
!pip install geopandas
!pip install Pillow








import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import time
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from shapely.geometry import mapping


from tensorflow.keras.callbacks import Callback
from google.cloud import storage
import os
import csv
from PIL import Image
import io
import json



# metrics_results = compute_metrics(new_ground_truth, new_predict)


# metrics_results

from matplotlib.colors import ListedColormap
class_colors = {
        1: ( 5, 5, 230),
        2: (190, 60, 15),
        3: (65, 240, 125),
        4: (105, 200, 95),
        5: ( 30, 115, 10),
        6: ( 255, 196, 34),
        7: (110, 85, 5),
        8: ( 235, 235, 220),
        9: (120, 216, 47),
        10: ( 84, 142, 128),
        11: ( 84, 142, 128),
        12: ( 84, 142, 128),
        13: ( 50, 255, 215),
        14: ( 50, 255, 215),
        15: ( 50, 255, 215),
        16: ( 193, 255, 0),
        17: ( 105, 200, 95),
        18: (105, 200, 95),
        19: ( 105, 200, 95),
        20: (193, 255, 0),
        21: ( 255, 50, 185),
        22: (255, 255, 255),
}

# Create a colormap using the class-color mapping
colors = [class_colors[i] for i in range(1, 23)]
normalized_colors_array = np.array([tuple(np.array(v) / 255.0) for v in class_colors.values()])

cmap_image = ListedColormap(normalized_colors_array)



class CustomMetricsCSVLogger(Callback):
    def __init__(self, filename, separator=',', append=True):
        super(CustomMetricsCSVLogger, self).__init__()
        self.filename = filename
        self.separator = separator
        self.append = append
        self.keys = None
        self.append_header = True
        self.max_epoch = 0  # Track the highest epoch number encountered
    def on_epoch_begin(self, epoch, logs=None):
        # Initialize min and max class-wise IOU at the beginning of each epoch
        self.min_class_wise_iou = 100
        self.max_class_wise_iou = 0

        # Check if the file exists in Cloud Storage, if not, create it
        if not self.file_exists():
            self.create_file()

    def file_exists(self):
        # Check if the file exists in Cloud Storage
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        return blob.exists()

    def create_file(self):
        # Create the file in Cloud Storage and write the header
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Write the header to the blob
        header = 'epoch,loss,val_loss,class_wise_iou,class_wise_dice_score,class_wise_accuracy,class_wise_precision,class_wise_recall,mean_iou,min_class_wise_iou,max_class_wise_iou\n'
        blob.upload_from_string(header)

    def parse_gcs_path(self, gcs_path):
        # Parse the Google Cloud Storage path to extract bucket name and blob name
        parts = gcs_path.replace('gs://', '').split('/')
        return parts[0], '/'.join(parts[1:])

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if self.keys is None:
            self.keys = sorted(logs.keys())

        # Extract class-wise IOU
        class_wise_iou = logs.get('class_wise_iou', 0.0)

        # Update min and max class-wise IOU
        self.min_class_wise_iou = min(self.min_class_wise_iou, class_wise_iou)
        self.max_class_wise_iou = max(self.max_class_wise_iou, class_wise_iou)

        # Append the row to the file in Cloud Storage
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Download existing content
        existing_content = blob.download_as_text() if blob.exists() else ""

        # Extract metrics values from logs
        metrics_values = [str(logs[key]) for key in ['loss', 'val_loss', 'class_wise_iou', 'class_wise_dice_score',
                                                    'class_wise_accuracy', 'class_wise_precision', 'class_wise_recall', 'mean_iou']]

        # Check if metrics for the current epoch already exist
        epoch_exists = any(f"{epoch}," in line for line in existing_content.split('\n'))

        # If the file is empty or epoch entry doesn't exist, append the metrics
        if not existing_content or not epoch_exists:
            updated_content = existing_content + f"{epoch},{','.join(metrics_values)},{self.min_class_wise_iou},{self.max_class_wise_iou}\n"
        else:
            # Get the maximum epoch number in the existing content
            max_existing_epoch = max(
                int(line.split(',')[0]) for line in existing_content.split('\n') if line.strip() and not line.startswith('epoch')
            )

            # Increment the epoch for the new entries
            updated_content = existing_content + f"{max_existing_epoch + 1},{','.join(metrics_values)},{self.min_class_wise_iou},{self.max_class_wise_iou}\n"

        # Update the highest epoch number
        self.max_epoch = max(self.max_epoch, epoch)

        # Upload updated content
        blob.upload_from_string(updated_content)





from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import os

class PredictSegmentationCallback(Callback):
    def __init__(self, test_image_path, output_save_path):
        super(PredictSegmentationCallback, self).__init__()
        self.test_image_path = test_image_path
        self.output_save_path = output_save_path
        self.last_predicted_array = None

    def parse_gcs_path(self, gcs_path):
        # Parse the Google Cloud Storage path to extract bucket name and blob name
        parts = gcs_path.replace('gs://', '').split('/')
        return parts[0], '/'.join(parts[1:])

    # def on_epoch_end(self, epoch, logs=None):
    #     # Call your prediction function
    #     self.last_predicted_array = prediction_function_img(self.test_image_path)
    #     normalized_array_image = self.last_predicted_array / (np.max(self.last_predicted_array) + 1e-10)
    #     # Display the predicted array with the specified colormap
    #     # plt.figure(figsize=(8, 4))
    #     # plt.imshow(normalized_array_image, cmap=cmap_image)  # Adjust the colormap as needed
    #     # plt.title(f'Predicted Array - Epoch {epoch}')
    #     # plt.show()

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 20 == 0:
            # Save the predicted array every 20 epochs
            # Convert NumPy array to PIL Image
            self.last_predicted_array = prediction_function_img(self.test_image_path)
            pil_image = Image.fromarray((self.last_predicted_array * 255).astype(np.uint8))

            # Apply the colormap to the PIL Image
            pil_image_colored = pil_image.convert('P', palette=Image.ADAPTIVE, colors=len(class_colors))
            pil_image_colored.putpalette(np.array(normalized_colors_array * 255, dtype=np.uint8).flatten())

            # Extract image name from the test_image_path for Google Cloud Storage
            image_name = os.path.basename(self.test_image_path)

            # Create the full GCS path for saving the predicted array as an image
            gcs_content_save_path = f'{self.output_save_path}/output_epoch_{epoch}_{image_name}.png'

            # Upload the image directly to GCS
            storage_client = storage.Client()
            bucket_name, blob_name = self.parse_gcs_path(gcs_content_save_path)
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            # Save the PIL Image to a BytesIO object
            image_io = io.BytesIO()
            pil_image_colored.save(image_io, format='PNG')

            image_io.seek(0)

            # Upload the image content to GCS
            blob.upload_from_file(image_io, content_type='image/png')  # Set the correct content type

            print(f'Predicted array saved at: {gcs_content_save_path}')
            





# imports================

import os
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from google.cloud import storage
# from python_files.model_file import get_model
import json
from datetime import datetime
# Necessary Functions------------------------------------------

feature_description = {
    "image": tf.io.VarLenFeature(tf.float32),
    "image_shape": tf.io.VarLenFeature(tf.int64),
    "label": tf.io.VarLenFeature(tf.float32),
    "label_shape": tf.io.VarLenFeature(tf.int64),
}

# def load_config(file_path):
#     with open(file_path, 'r') as f:
#         config = json.load(f)
#     return config

# config = load_config('./python_files/config.json')

def parse(serialized_examples):
    return tf.io.parse_example(serialized_examples, feature_description)


def create_dataset(input_directory, image_names=[]):
    if len(image_names)==0:
        tfrecord_files = [
        f"{input_directory}{file}"
        for file in tf.io.gfile.listdir(input_directory)
        if file.endswith(".tfrecord")
    ]
    else:
        names = [name.split(".")[0]+"_tfrecord" for name in image_names]
        print(names)
        tfrecord_files = [
            f"{input_directory}{file}"
            for file in tf.io.gfile.listdir(input_directory)
            if file.endswith(".tfrecord") and os.path.basename(file).rsplit('.', 1)[0] in names
        ]
    dataset = tf.data.TFRecordDataset(tfrecord_files).map(parse, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset


# # pre-processing functions
# def bandwise_normalize(input_tensor, epsilon=1e-8):
#     # Convert the input_tensor to a float32 type
#     input_tensor = tf.cast(input_tensor, tf.float32)

#     # Calculate the minimum and maximum values along the channel axis
#     min_val = tf.reduce_min(input_tensor, axis=2, keepdims=True)
#     max_val = tf.reduce_max(input_tensor, axis=2, keepdims=True)

#     # Check for potential numerical instability
#     denom = max_val - min_val
#     denom = tf.where(tf.abs(denom) < epsilon, epsilon, denom)

#     # Normalize the tensor band-wise to the range [0, 1]
#     normalized_tensor = (input_tensor - min_val) / denom

#     return normalized_tensor


# def pad_to_multiple(image, TILE_HT, TILE_WD):
#     # Get the current dimensions
#     height, width, channels = image.shape

#     # Calculate the target dimensions
#     target_height = tf.cast(tf.math.ceil(height / TILE_HT) * TILE_HT, tf.int32)
#     target_width = tf.cast(tf.math.ceil(width / TILE_WD) * TILE_WD, tf.int32)

#     # Calculate the amount of padding
#     pad_height = target_height - height
#     pad_width = target_width - width

#     # Pad the image
#     padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)

#     return padded_image


# def tile_image(fullimg, CHANNELS, TILE_HT, TILE_WD):
#     fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
#     images = tf.expand_dims(fullimg, axis=0)
#     tiles = tf.image.extract_patches(
#         images=images,
#         sizes=[1, TILE_HT, TILE_WD, 1],
#         strides=[1, TILE_HT, TILE_WD, 1],
#         rates=[1, 1, 1, 1],
#         padding="VALID",
#     )

#     tiles = tf.squeeze(tiles, axis=0)
#     nrows = tiles.shape[0]
#     ncols = tiles.shape[1]
#     tiles = tf.reshape(tiles, [nrows, ncols, TILE_HT, TILE_WD, CHANNELS])
#     return tiles


# def sampling(label_image, threshold_percentage=99.9):
#     num_zeros = tf.reduce_sum(
#         tf.cast(tf.equal(label_image, 0), tf.float32), axis=[2, 3, 4]
#     )

#     # Calculate the total number of elements in each patch
#     total_elements = tf.cast(tf.reduce_prod(tf.shape(label_image)[2:]), tf.float32)

#     # Calculate the percentage of zeros in each patch
#     percentage_zeros = (num_zeros / total_elements) * 100.0

#     boolean_mask = percentage_zeros <= threshold_percentage
#     # Apply the threshold logic
#     sampled_tensor = tf.cast(percentage_zeros >= threshold_percentage, tf.int32)
#     return boolean_mask, sampled_tensor


# def one_hot_encoding(label_tensor):
#     # Assuming your pixel values are float labels
#     float_labels = tf.squeeze(
#         label_tensor, axis=-1
#     )  # Assuming channel dimension is the last one

#     # Determine the number of classes dynamically
#     num_classes = tf.cast(tf.reduce_max(float_labels) + 1, tf.int32)

#     # One-hot encode each image
#     one_hot_encoded_images = tf.one_hot(
#         tf.dtypes.cast(float_labels, tf.int32), depth=num_classes
#     )

#     # Print the shape of the resulting tensor and the number of classes
#     # print("Shape of one-hot encoded images:", one_hot_encoded_images.shape)
#     # print("Number of classes:", num_classes)

#     return one_hot_encoded_images



# # @tf.function
# def parse_example(parsed_example, patch_height, patch_width, threshold_percentage, image_channels, label_channels):
#     image_shape = tf.sparse.to_dense(parsed_example["image_shape"])
#     image = tf.reshape(tf.sparse.to_dense(parsed_example["image"]), image_shape)
#     label_shape = tf.sparse.to_dense(parsed_example["label_shape"])
#     label = tf.reshape(tf.sparse.to_dense(parsed_example["label"]), label_shape)
#     print(image_shape)
#     print(parsed_example["image_shape"])
#     # image normalization
#     image = bandwise_normalize(image)

#     # image and label patching
#     image_patches = tile_image(image, image_channels, patch_height, patch_width)
#     label_patches = tile_image(label, label_channels, patch_height, patch_width)

#     # sampling
#     sampled_mask, _ = sampling(label_patches, threshold_percentage)
#     sampled_image_patches = tf.boolean_mask(image_patches, sampled_mask)
#     sampled_label_patches = tf.boolean_mask(label_patches, sampled_mask)

#     # one-hot encoding
#     sampled_label_patches = one_hot_encoding(sampled_label_patches)

#     return sampled_image_patches, sampled_label_patches

# def parsing(dataset, patch_height, patch_width, threshold_percentage, image_channels, label_channels):
#     image_patch_tensors_list = []
#     label_patch_tensors_list = []
#     for parsed_example in dataset:
#         sampled_image_patches, sampled_label_patches = parse_example(parsed_example, patch_height, patch_width, threshold_percentage, image_channels, label_channels)
#         image_patch_tensors_list.append(sampled_image_patches)
#         label_patch_tensors_list.append(sampled_label_patches)

#     return image_patch_tensors_list, label_patch_tensors_list
import tensorflow as tf

# Custom pre-processing functions
@tf.function
def bandwise_normalize(input_tensor, epsilon=1e-8):
    input_tensor = tf.cast(input_tensor, tf.float32)
    min_val = tf.reduce_min(input_tensor, axis=2, keepdims=True)
    max_val = tf.reduce_max(input_tensor, axis=2, keepdims=True)
    denom = max_val - min_val
    denom = tf.where(tf.abs(denom) < epsilon, epsilon, denom)
    normalized_tensor = (input_tensor - min_val) / denom
    return normalized_tensor

@tf.function
# def pad_to_multiple(image, TILE_HT, TILE_WD):
#     height, width, channels = image.shape
#     target_height = tf.cast(tf.math.ceil(height / TILE_HT) * TILE_HT, tf.int32)
#     target_width = tf.cast(tf.math.ceil(width / TILE_WD) * TILE_WD, tf.int32)
#     padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
#     return padded_image
def pad_to_multiple(image, TILE_HT, TILE_WD):
    target_height = tf.cast(tf.math.ceil(tf.shape(image)[0] / TILE_HT) * TILE_HT, tf.int32)
    target_width = tf.cast(tf.math.ceil(tf.shape(image)[1] / TILE_WD) * TILE_WD, tf.int32)
    padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
    return padded_image

@tf.function
def tile_image(fullimg, CHANNELS, TILE_HT, TILE_WD):
    fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
    images = tf.expand_dims(fullimg, axis=0)
    tiles = tf.image.extract_patches(
        images=images,
        sizes=[1, TILE_HT, TILE_WD, 1],
        strides=[1, TILE_HT, TILE_WD, 1],
        rates=[1, 1, 1, 1],
        padding="VALID",
    )
    tiles = tf.squeeze(tiles, axis=0)
    # Cast elements to a supported data type
    tiles = tf.cast(tiles, tf.float32)
    
    # Reshape without specifying batch size dimensions
    tiles_shape = tf.shape(tiles)
    tiles = tf.reshape(tiles, [tiles_shape[0], tiles_shape[1], TILE_HT, TILE_WD, CHANNELS])
    return tiles

@tf.function
def sampling(label_image, threshold_percentage=99.9):
    num_zeros = tf.reduce_sum(tf.cast(tf.equal(label_image, 0), tf.float32), axis=[2, 3, 4])
    total_elements = tf.cast(tf.reduce_prod(tf.shape(label_image)[2:]), tf.float32)
    percentage_zeros = (num_zeros / total_elements) * 100.0
    boolean_mask = percentage_zeros <= threshold_percentage
    return boolean_mask, tf.cast(percentage_zeros >= threshold_percentage, tf.int32)

@tf.function
def one_hot_encoding(label_tensor):
    float_labels = tf.squeeze(label_tensor, axis=-1)
    num_classes = tf.cast(tf.reduce_max(float_labels) + 1, tf.int32)
    one_hot_encoded_images = tf.one_hot(tf.dtypes.cast(float_labels, tf.int32), depth=num_classes)
    return one_hot_encoded_images

# Optimized parsing function using tf.data.Dataset
def parsing(dataset, patch_height, patch_width, threshold_percentage, image_channels, label_channels):
    def parse_example(parsed_example):
        image_shape = tf.sparse.to_dense(parsed_example["image_shape"])
        image = tf.reshape(tf.sparse.to_dense(parsed_example["image"]), image_shape)
        label_shape = tf.sparse.to_dense(parsed_example["label_shape"])
        label = tf.reshape(tf.sparse.to_dense(parsed_example["label"]), label_shape)
        
        image = bandwise_normalize(image)
        image_patches = tile_image(image, image_channels, patch_height, patch_width)
        label_patches = tile_image(label, label_channels, patch_height, patch_width)
        
        sampled_mask, _ = sampling(label_patches, threshold_percentage)
        sampled_image_patches = tf.boolean_mask(image_patches, sampled_mask)
        sampled_label_patches = tf.boolean_mask(label_patches, sampled_mask)
        
        sampled_label_patches = one_hot_encoding(sampled_label_patches)
        
        return sampled_image_patches, sampled_label_patches

    # Apply the parse_example function to each element in the dataset
    parsed_dataset = dataset.map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    
    image_patch_tensors_list = []
    label_patch_tensors_list = []

    for image_patches, label_patches in parsed_dataset:

        # Append to lists
        image_patch_tensors_list.append(image_patches)
        label_patch_tensors_list.append(label_patches)
        
    return image_patch_tensors_list, label_patch_tensors_list




# Modeling--------------------------------------------------

def train_test_datasets(
    input_directory,
    patch_height,
    patch_width,
    image_channels,
    label_channels,
    threshold_percentage,
    batch_size,
):
    dataset = create_dataset(input_directory)
    image_patch_tensors_list, label_patch_tensors_list = parsing(
        dataset=dataset,
        patch_height=patch_height,
        patch_width=patch_width,
        image_channels=image_channels,
        label_channels=label_channels,
        threshold_percentage=threshold_percentage,
    )

    # Combine images and labels from different pairs
    combined_images = tf.concat(image_patch_tensors_list, axis=0)
    combined_labels = tf.concat(label_patch_tensors_list, axis=0)

    # Shuffle the combined data
    combined_dataset = tf.data.Dataset.from_tensor_slices(
        (combined_images, combined_labels)
    )
    combined_dataset = combined_dataset.shuffle(buffer_size=combined_images.shape[0])

    # Split the combined dataset into training and validation sets
    train_size = int(0.8 * combined_images.shape[0])
    train_dataset = combined_dataset.take(train_size)
    val_dataset = combined_dataset.skip(train_size)

    # Batch the data using TensorFlow's Dataset API
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)

    return train_dataset, val_dataset





def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client(project="gislogics")
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print("File {} uploaded to {}.".format(source_file_name, destination_blob_name))

    
    
    

def load_config(file_path):
    with open(file_path, 'r') as f:
        config = json.load(f)
    return config








# Modeling--------------------------------------------------
config = load_config('./pipeline_scripts/config.json')
from google.cloud import storage
from tensorflow.keras.models import save_model, load_model
from tensorflow.python.lib.io import file_io
# Convert img_size to a tuple
config["img_size"] = tuple(config["img_size"])

# Now you can access the config values normally
model_path = config.get("model_path")
model_name = config.get("model_name")
test_image_path = config.get("test_image_path")
img_size = config.get("img_size")
num_bands = config.get("num_bands")
num_classes = config.get("num_classes")
gcs_path = config.get("google_storage_path")


# Load the model if a checkpoint exists
# Load the model if a checkpoint exists
import gcsfs

fs = gcsfs.GCSFileSystem(project='gislogics')  # Set up GCS connection

if tf.io.gfile.exists(model_path):  # Accurately check for file existence in GCS
    # Load existing model
    model = load_model(model_path)  # Handle GCS authentication here if needed
    print("Loaded model from checkpoint")
else:
    # If no checkpoint exists, create a new model
    model = get_model(
        img_size=img_size, num_classes=num_classes, num_bands=num_bands
    )
    model.compile(optimizer="adam", loss="categorical_crossentropy")
    print("Created a new model")

    # # Save the new model to the model_path in GCS
    save_model(model, model_path)
    print("Saved new model to:", model_path)

    
    
    
    
# Continue training
from pipeline_scripts.eval import *
from pipeline_scripts.prediction import *

def train(**kwargs):
    input_directory = kwargs.get("input_directory")
    threshold_percentage = kwargs.get("threshold_percentage")
    image_channels = kwargs.get("image_channels")
    label_channels = kwargs.get("label_channels")
    patch_height = kwargs.get("patch_height")
    patch_width = kwargs.get("patch_width")
    batch_size = kwargs.get("batch_size")
    num_classes = kwargs.get("num_classes")
    
    bucket_name = kwargs.get("bucket_name")
    img_size = (patch_height, patch_width)
    

    



    # get the train and test datasets
    train_dataset, val_dataset = train_test_datasets(
        input_directory,
        patch_height,
        patch_width,
        image_channels,
        label_channels,
        threshold_percentage,
        batch_size,
    )
    
    print("Train and Valid datasets are created")

    # create img_size
    model = get_model(
        img_size= img_size, 
        num_classes=num_classes, 
        num_bands=image_channels
    )
    
    
    # compilation of model, with custom metric
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy", 
        metrics=[np.mean(compute_metrics(new_ground_truth, new_predict)[0])]
    )

    
    # Early stopping after 5 epochs 
    early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,  
    restore_best_weights=True,  
    verbose=1  
    )
    
    # Agroforestry Class
    Agroforestry = 18
    
    # including custom metrics in callbacks
    custom_metrics_callback = keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs: logs.update({
        "class_wise_iou": compute_metrics(new_ground_truth, new_predict)[0][Agroforestry],
        "class_wise_dice_score": compute_metrics(new_ground_truth, new_predict)[1][Agroforestry],
        "class_wise_accuracy": compute_metrics(new_ground_truth, new_predict)[2][Agroforestry],
        "class_wise_precision": compute_metrics(new_ground_truth, new_predict)[3][Agroforestry],
        "class_wise_recall": compute_metrics(new_ground_truth, new_predict)[4][Agroforestry],
        "mean_iou": compute_metrics(new_ground_truth, new_predict)[5],
        "min_class_wise_iou": np.min(metrics_results[0]),
        "max_class_wise_iou": np.max(metrics_results[0]),
        "epoch": epoch,
        "loss": logs["loss"],
        "val_loss": logs["val_loss"]}))

    
    # callbacks and logging
    csv_logger = keras.callbacks.CSVLogger(
    input_directory + "logs/" + f"training_logs_{model_name}.csv",
    append=True
    )

    custom_metrics_csv_logger = CustomMetricsCSVLogger(
        input_params["input_directory"] + "logs/" + f"training_logs_{model_name}.csv",
        append=True
    )
    
    
    log_dir = "gs://tf_records_bucket/tf_records/Untitled Folder/logs/"  # Specify the directory to save logs
    tensorboard_callback = TensorBoard(log_dir=log_dir)

    output_of_image = "gs://tf_records_bucket/tf_records/Untitled Folder/output"
    
    # Combine all callbacks
    all_callbacks = [
        keras.callbacks.ModelCheckpoint(model_path + model_name, save_best_only=False),
        tensorboard_callback,
        custom_metrics_callback,
        custom_metrics_csv_logger,  # Add the custom_metrics_csv_logger here
        # early_stopping,
        PredictSegmentationCallback(test_image_path, output_of_image)
    ]

    model_history = model.fit(
        train_dataset,
        epochs=3,
        callbacks=all_callbacks,
        batch_size=32,
        validation_data=val_dataset,
    )

    if early_stopping.stopped_epoch > 0:
        print(f"Training stopped at epoch {early_stopping.stopped_epoch} due to early stopping.")
    else:
        print("Training completed all epochs.")
    # Save the model after training
    # model.save(model_path + model_name)
    # print("Model saved locally")

    upload_blob(gcs_path)
    print("Uploaded to cloud storage successfully")



if __name__ == "__main__":
    input_params = {
        "input_directory": "gs://tf_records_bucket/tf_records/Untitled Folder/",  # make sure / is there at the end,
        "bucket_name": "tf_records_bucket",
        "threshold_percentage": 99.9,
        "image_channels": 8,  # 8 bands images as input
        "label_channels": 1,
        "patch_height": 8,
        "patch_width": 8,
        "batch_size": 32,
        "num_classes": 23,
        "model_path": "trained_model/",
    }  # makesure there is a slash at the end of the path  # Choose an appropriate batch size

    train(**input_params)







# ! tensorboard --logdir gs://tf_records_bucket/tf_records/Untitled\ Folder/logs/ 









  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      Traceback (most recent call last):
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 126, in _parse_requirement_details
      

ValueError: in user code:

    File "/tmp/ipykernel_1/2447842573.py", line 483, in parse_example  *
        image_patches = tile_image(image, image_channels, patch_height, patch_width)
    File "/home/jupyter/eval_and_pred/pipeline_scripts/prediction.py", line 135, in tile_image  *
        fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
    File "/home/jupyter/eval_and_pred/pipeline_scripts/prediction.py", line 117, in pad_to_multiple  *
        height, width, channels = image.shape

    ValueError: Cannot iterate over a shape with unknown rank.


In [10]:
# input_params = {
#         "input_directory": "gs://tf_records_bucket/tf_records/",  # make sure / is there at the end,
#         "image_names": [], # "area2_0530_2022_8bands.tif", "area2_0909_2023_composite.tif", "area2_0911_2023_8bands.tif"
#         "bucket_name": "tf_records_bucket",
#         "threshold_percentage": 99.9,
#         "image_channels": 8,  # 8 bands images as input
#         "label_channels": 1,
#         "patch_height": 8,
#         "patch_width": 8,
#         "batch_size": 32, 
#         # "model_path": "trained_model/",
#     }  # makesure there is a slash at the end of the path  # Choose an appropriate batch size

# train(**input_params)


2024-03-07 14:34:22.000097: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [11]
	 [[{{node Placeholder/_0}}]]
2024-03-07 14:34:22.000352: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [11]
	 [[{{node Placeholder/_0}}]]


Train and Valid datasets are created


In [10]:
# %cd /home/jupyter/eval_and_pred

/home/jupyter/eval_and_pred


# **Leftover Code from EVAL and PRED**

In [18]:

!pip install rasterio
!pip install geopandas
!pip install Pillow














import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import time
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from shapely.geometry import mapping
from datetime import datetime


from tensorflow.keras.callbacks import Callback
from google.cloud import storage
import os
import csv
from PIL import Image
import io
import json



# metrics_results = compute_metrics(new_ground_truth, new_predict)


# metrics_results

from matplotlib.colors import ListedColormap
class_colors = {
        1: ( 5, 5, 230),
        2: (190, 60, 15),
        3: (65, 240, 125),
        4: (105, 200, 95),
        5: ( 30, 115, 10),
        6: ( 255, 196, 34),
        7: (110, 85, 5),
        8: ( 235, 235, 220),
        9: (120, 216, 47),
        10: ( 84, 142, 128),
        11: ( 84, 142, 128),
        12: ( 84, 142, 128),
        13: ( 50, 255, 215),
        14: ( 50, 255, 215),
        15: ( 50, 255, 215),
        16: ( 193, 255, 0),
        17: ( 105, 200, 95),
        18: (105, 200, 95),
        19: ( 105, 200, 95),
        20: (193, 255, 0),
        21: ( 255, 50, 185),
        22: (255, 255, 255),
}

# Create a colormap using the class-color mapping
colors = [class_colors[i] for i in range(1, 23)]
normalized_colors_array = np.array([tuple(np.array(v) / 255.0) for v in class_colors.values()])

cmap_image = ListedColormap(normalized_colors_array)



class CustomMetricsCSVLogger(Callback):
    def __init__(self, filename, separator=',', append=True):
        super(CustomMetricsCSVLogger, self).__init__()
        self.filename = filename
        self.separator = separator
        self.append = append
        self.keys = None
        self.append_header = True
        self.max_epoch = 0  # Track the highest epoch number encountered
    def on_epoch_begin(self, epoch, logs=None):
        # Initialize min and max class-wise IOU at the beginning of each epoch
        self.min_class_wise_iou = 100
        self.max_class_wise_iou = 0

        # Check if the file exists in Cloud Storage, if not, create it
        if not self.file_exists():
            self.create_file()

    def file_exists(self):
        # Check if the file exists in Cloud Storage
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        return blob.exists()

    def create_file(self):
        # Create the file in Cloud Storage and write the header
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Write the header to the blob
        header = 'epoch,loss,val_loss,class_wise_iou,class_wise_dice_score,class_wise_accuracy,class_wise_precision,class_wise_recall,mean_iou,min_class_wise_iou,max_class_wise_iou\n'
        blob.upload_from_string(header)

    def parse_gcs_path(self, gcs_path):
        # Parse the Google Cloud Storage path to extract bucket name and blob name
        parts = gcs_path.replace('gs://', '').split('/')
        return parts[0], '/'.join(parts[1:])

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if self.keys is None:
            self.keys = sorted(logs.keys())

        # Extract class-wise IOU
        class_wise_iou = logs.get('class_wise_iou', 0.0)

        # Update min and max class-wise IOU
        self.min_class_wise_iou = min(self.min_class_wise_iou, class_wise_iou)
        self.max_class_wise_iou = max(self.max_class_wise_iou, class_wise_iou)

        # Append the row to the file in Cloud Storage
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Download existing content
        existing_content = blob.download_as_text() if blob.exists() else ""

        # Extract metrics values from logs
        metrics_values = [str(logs[key]) for key in ['loss', 'val_loss', 'class_wise_iou', 'class_wise_dice_score',
                                                    'class_wise_accuracy', 'class_wise_precision', 'class_wise_recall', 'mean_iou']]

        # Check if metrics for the current epoch already exist
        epoch_exists = any(f"{epoch}," in line for line in existing_content.split('\n'))

        # If the file is empty or epoch entry doesn't exist, append the metrics
        if not existing_content or not epoch_exists:
            updated_content = existing_content + f"{epoch},{','.join(metrics_values)},{self.min_class_wise_iou},{self.max_class_wise_iou}\n"
        else:
            # Get the maximum epoch number in the existing content
            max_existing_epoch = max(
                int(line.split(',')[0]) for line in existing_content.split('\n') if line.strip() and not line.startswith('epoch')
            )

            # Increment the epoch for the new entries
            updated_content = existing_content + f"{max_existing_epoch + 1},{','.join(metrics_values)},{self.min_class_wise_iou},{self.max_class_wise_iou}\n"

        # Update the highest epoch number
        self.max_epoch = max(self.max_epoch, epoch)

        # Upload updated content
        blob.upload_from_string(updated_content)





from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import os

class PredictSegmentationCallback(Callback):
    def __init__(self, test_image_path, output_save_path):
        super(PredictSegmentationCallback, self).__init__()
        self.test_image_path = test_image_path
        self.output_save_path = output_save_path
        self.last_predicted_array = None

    def parse_gcs_path(self, gcs_path):
        # Parse the Google Cloud Storage path to extract bucket name and blob name
        parts = gcs_path.replace('gs://', '').split('/')
        return parts[0], '/'.join(parts[1:])

    # def on_epoch_end(self, epoch, logs=None):
    #     # Call your prediction function
    #     self.last_predicted_array = prediction_function_img(self.test_image_path)
    #     normalized_array_image = self.last_predicted_array / (np.max(self.last_predicted_array) + 1e-10)
    #     # Display the predicted array with the specified colormap
    #     # plt.figure(figsize=(8, 4))
    #     # plt.imshow(normalized_array_image, cmap=cmap_image)  # Adjust the colormap as needed
    #     # plt.title(f'Predicted Array - Epoch {epoch}')
    #     # plt.show()

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 20 == 0:
            # Save the predicted array every 20 epochs
            # Convert NumPy array to PIL Image
            self.last_predicted_array = prediction_function_img(self.test_image_path)
            pil_image = Image.fromarray((self.last_predicted_array * 255).astype(np.uint8))

            # Apply the colormap to the PIL Image
            pil_image_colored = pil_image.convert('P', palette=Image.ADAPTIVE, colors=len(class_colors))
            pil_image_colored.putpalette(np.array(normalized_colors_array * 255, dtype=np.uint8).flatten())

            # Extract image name from the test_image_path for Google Cloud Storage
            image_name = os.path.basename(self.test_image_path)

            # Create the full GCS path for saving the predicted array as an image
            gcs_content_save_path = f'{self.output_save_path}/output_epoch_{epoch}_{image_name}.png'

            # Upload the image directly to GCS
            storage_client = storage.Client()
            bucket_name, blob_name = self.parse_gcs_path(gcs_content_save_path)
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            # Save the PIL Image to a BytesIO object
            image_io = io.BytesIO()
            pil_image_colored.save(image_io, format='PNG')

            image_io.seek(0)

            # Upload the image content to GCS
            blob.upload_from_file(image_io, content_type='image/png')  # Set the correct content type

            print(f'Predicted array saved at: {gcs_content_save_path}')
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
# imports================
# imports================

import os
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from google.cloud import storage
from pipeline_scripts.model_file import get_model

# Necessary Functions------------------------------------------

feature_description = {
    "image": tf.io.VarLenFeature(tf.float32),
    "image_shape": tf.io.VarLenFeature(tf.int64),
    "label": tf.io.VarLenFeature(tf.float32),
    "label_shape": tf.io.VarLenFeature(tf.int64),
    # "label": tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
    # "label_shape": tf.io.FixedLenFeature([], tf.int64, default_value=0),
}


def parse(serialized_examples):
    return tf.io.parse_example(serialized_examples, feature_description)


def create_dataset(input_directory, image_names=[]):
    if len(image_names)==0:
        tfrecord_files = [
        f"{input_directory}{file}"
        for file in tf.io.gfile.listdir(input_directory)
        if file.endswith(".tfrecord")
    ]
    else:
        names = [name.split(".")[0]+"_tfrecord" for name in image_names]
        print(names)
        tfrecord_files = [
            f"{input_directory}{file}"
            for file in tf.io.gfile.listdir(input_directory)
            if file.endswith(".tfrecord") and os.path.basename(file).rsplit('.', 1)[0] in names
        ]
    print(f"TFRecords List: {tfrecord_files}")
    dataset = tf.data.TFRecordDataset(tfrecord_files).map(parse, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset



# pre-processing functions
@tf.function
def bandwise_normalize(input_tensor, epsilon=1e-8):
    input_tensor = tf.cast(input_tensor, tf.float32)
    min_val = tf.reduce_min(input_tensor, axis=2, keepdims=True)
    max_val = tf.reduce_max(input_tensor, axis=2, keepdims=True)
    denom = max_val - min_val
    denom = tf.where(tf.abs(denom) < epsilon, epsilon, denom)
    normalized_tensor = (input_tensor - min_val) / denom
    return normalized_tensor

@tf.function
def pad_to_multiple(image, TILE_HT, TILE_WD):
    if image.shape.rank is None:  # Check for unknown rank
        try:
            image = tf.expand_dims(image, axis=-1)  # Add a channel dimension if missing
        except ValueError:
            raise ValueError("Image has an invalid rank") from None
    target_height = tf.cast(tf.math.ceil(tf.shape(image)[0] / TILE_HT) * TILE_HT, tf.int32)
    target_width = tf.cast(tf.math.ceil(tf.shape(image)[1] / TILE_WD) * TILE_WD, tf.int32)
    padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
    return padded_image

@tf.function
def tile_image(fullimg, CHANNELS, TILE_HT, TILE_WD):
    fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
    images = tf.expand_dims(fullimg, axis=0)
    tiles = tf.image.extract_patches(
        images=images,
        sizes=[1, TILE_HT, TILE_WD, 1],
        strides=[1, TILE_HT, TILE_WD, 1],
        rates=[1, 1, 1, 1],
        padding="VALID",
    )
    tiles = tf.squeeze(tiles, axis=0)
    # Cast elements to a supported data type
    tiles = tf.cast(tiles, tf.float32)
    
    # Reshape without specifying batch size dimensions
    tiles_shape = tf.shape(tiles)
    tiles = tf.reshape(tiles, [tiles_shape[0], tiles_shape[1], TILE_HT, TILE_WD, CHANNELS])
    print("Output Tensor Rank:", tf.rank(tiles))
    return tiles

@tf.function
def sampling(label_image, threshold_percentage=99.9):
    # num_zeros = tf.reduce_sum(tf.cast(tf.equal(label_image, 0), tf.float32), axis=[2, 3, 4])
    num_zeros = tf.reduce_sum(tf.cast(tf.equal(label_image, tf.constant(0, dtype=tf.float32)), tf.float32), axis=[2, 3, 4])
    total_elements = tf.cast(tf.reduce_prod(tf.shape(label_image)[2:]), tf.float32)
    percentage_zeros = (num_zeros / total_elements) * 100.0
    boolean_mask = percentage_zeros <= threshold_percentage
    return boolean_mask, tf.cast(percentage_zeros >= threshold_percentage, tf.int32)

@tf.function
def one_hot_encoding(label_tensor):
    float_labels = tf.squeeze(label_tensor, axis=-1)
    num_classes = tf.cast(tf.reduce_max(float_labels) + 1, tf.int32)
    one_hot_encoded_images = tf.one_hot(tf.dtypes.cast(float_labels, tf.int32), depth=num_classes)
    return one_hot_encoded_images

# Optimized parsing function using tf.data.Dataset
def parsing(dataset, patch_height, patch_width, threshold_percentage, image_channels, label_channels):
    def parse_example(parsed_example):
        image_shape = tf.sparse.to_dense(parsed_example["image_shape"])
        print("parsed_example", parsed_example)
        print('parsed_example["image_shape"]', parsed_example["image_shape"])
        print("image shape in parse example", image_shape)
        image = tf.reshape(tf.sparse.to_dense(parsed_example["image"]), tf.cast(image_shape, tf.int64))
        label_shape = tf.sparse.to_dense(parsed_example["label_shape"])
        label = tf.reshape(tf.sparse.to_dense(parsed_example["label"]), tf.cast(label_shape, tf.int64))
        
        image = bandwise_normalize(image)
        print(f"Bandwise Norm Image Shape: {image.shape}")
        image_patches = tile_image(image, image_channels, patch_height, patch_width)
        label_patches = tile_image(label, label_channels, patch_height, patch_width)
        
        sampled_mask, _ = sampling(label_patches, threshold_percentage)
        sampled_image_patches = tf.boolean_mask(image_patches, sampled_mask)
        sampled_label_patches = tf.boolean_mask(label_patches, sampled_mask)
        
        sampled_label_patches = one_hot_encoding(sampled_label_patches)
        
        return sampled_image_patches, sampled_label_patches
    
    for d in dataset:
        print(f"Dataset: {d}")

      # Preprocessing within parsing loop for efficient memory usage
    image_patch_tensors = []
    label_patch_tensors = []
    for parsed_data in dataset:
        sampled_image_patches, sampled_label_patches = parse_example(parsed_data)
        image_patch_tensors.append(sampled_image_patches)
        label_patch_tensors.append(sampled_label_patches)

      # Stack tensors for final output
    # image_patch_tensors = tf.stack(image_patch_tensors)
    # label_patch_tensors = tf.stack(label_patch_tensors)

    return image_patch_tensors, label_patch_tensors
#     # Apply the parse_example function to each element in the dataset
#     parsed_dataset = dataset.map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    
#     image_patch_tensors_list = []
#     label_patch_tensors_list = []

#     for image_patches, label_patches in parsed_dataset:

#         # Append to lists
#         image_patch_tensors_list.append(image_patches)
#         label_patch_tensors_list.append(label_patches)
        
#     return image_patch_tensors_list, label_patch_tensors_list



def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client(project="gislogics")
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print("File {} uploaded to {}.".format(source_file_name, destination_blob_name))

    
def train_test_datasets(
    input_directory,
    patch_height,
    patch_width,
    image_channels,
    label_channels,
    threshold_percentage,
    batch_size,
    image_names
):
    dataset = create_dataset(input_directory, image_names)
    image_patch_tensors_list, label_patch_tensors_list = parsing(
        dataset=dataset,
        patch_height=patch_height,
        patch_width=patch_width,
        image_channels=image_channels,
        label_channels=label_channels,
        threshold_percentage=threshold_percentage,
    )

    # Combine images and labels from different pairs
    combined_images = tf.concat(image_patch_tensors_list, axis=0)
    combined_labels = tf.concat(label_patch_tensors_list, axis=0)

    # Shuffle the combined data
    combined_dataset = tf.data.Dataset.from_tensor_slices(
        (combined_images, combined_labels)
    )
    combined_dataset = combined_dataset.shuffle(buffer_size=combined_images.shape[0])

    # Split the combined dataset into training and validation sets
    train_size = int(0.8 * combined_images.shape[0])
    train_dataset = combined_dataset.take(train_size)
    val_dataset = combined_dataset.skip(train_size)

    # Batch the data using TensorFlow's Dataset API
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)

    return train_dataset, val_dataset
    

def load_config(file_path):
    with open(file_path, 'r') as f:
        config = json.load(f)
    return config








# Modeling--------------------------------------------------
config = load_config('./pipeline_scripts/config.json')
from google.cloud import storage
from tensorflow.keras.models import save_model, load_model
from tensorflow.python.lib.io import file_io
# Convert img_size to a tuple
config["img_size"] = tuple(config["img_size"])

# Now you can access the config values normally
model_path = config.get("model_path")
model_name = config.get("model_name")
test_image_path = config.get("test_image_path")
img_size = config.get("img_size")
num_bands = config.get("num_bands")
num_classes = config.get("num_classes")
gcs_path = config.get("google_storage_path")
gcs_tfrecords = config.get("gcs_tfrecords")


# Load the model if a checkpoint exists
# Load the model if a checkpoint exists
import gcsfs

fs = gcsfs.GCSFileSystem(project='gislogics')  # Set up GCS connection

if tf.io.gfile.exists(model_path):  # Accurately check for file existence in GCS
    # Load existing model
    model = load_model(model_path)  # Handle GCS authentication here if needed
    print("Loaded model from checkpoint")
else:
    # If no checkpoint exists, create a new model
    model = get_model(
        img_size=img_size, num_classes=num_classes, num_bands=num_bands
    )
    model.compile(optimizer="adam", loss="categorical_crossentropy")
    print("Created a new model")

    # # Save the new model to the model_path in GCS
    save_model(model, model_path)
    print("Saved new model to:", model_path)

    
    
    
    
# Continue training
from pipeline_scripts.eval import *
from pipeline_scripts.prediction import *

def train(**kwargs):
    input_directory = kwargs.get("input_directory")
    threshold_percentage = kwargs.get("threshold_percentage")
    image_channels = kwargs.get("image_channels")
    label_channels = kwargs.get("label_channels")
    patch_height = kwargs.get("patch_height")
    patch_width = kwargs.get("patch_width")
    batch_size = kwargs.get("batch_size")
    num_classes = kwargs.get("num_classes")  
    bucket_name = kwargs.get("bucket_name")

    image_names = kwargs.get("image_names")
    # img_size = (patch_height, patch_width)
    
    

    # get the train and test datasets
    train_dataset, val_dataset = train_test_datasets(
        gcs_tfrecords,
        patch_height,
        patch_width,
        image_channels,
        label_channels,
        threshold_percentage,
        batch_size,
        image_names
    )
    print("Train and Valid datasets are created")

    # create img_size
    model = get_model(
        img_size= img_size, 
        num_classes=num_classes, 
        num_bands=image_channels
    )
    
    
    # compilation of model, with custom metric
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy", 
        metrics=[np.mean(compute_metrics(new_ground_truth, new_predict)[0])]
    )

    
    # Early stopping after 5 epochs 
    early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,  
    restore_best_weights=True,  
    verbose=1  
    )
    
    # Agroforestry Class
    Agroforestry = 18
    
    # including custom metrics in callbacks
    custom_metrics_callback = keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs: logs.update({
        "class_wise_iou": compute_metrics(new_ground_truth, new_predict)[0][Agroforestry],
        "class_wise_dice_score": compute_metrics(new_ground_truth, new_predict)[1][Agroforestry],
        "class_wise_accuracy": compute_metrics(new_ground_truth, new_predict)[2][Agroforestry],
        "class_wise_precision": compute_metrics(new_ground_truth, new_predict)[3][Agroforestry],
        "class_wise_recall": compute_metrics(new_ground_truth, new_predict)[4][Agroforestry],
        "mean_iou": compute_metrics(new_ground_truth, new_predict)[5],
        "min_class_wise_iou": np.min(metrics_results[0]),
        "max_class_wise_iou": np.max(metrics_results[0]),
        "epoch": epoch,
        "loss": logs["loss"],
        "val_loss": logs["val_loss"]}))

    
    # callbacks and logging
    csv_logger = keras.callbacks.CSVLogger(
    input_directory + "logs/" + f"training_logs_{model_name}.csv",
    append=True
    )

    custom_metrics_csv_logger = CustomMetricsCSVLogger(
        input_params["input_directory"] + "logs/" + f"training_logs_{model_name}.csv",
        append=True
    )
    
    
    log_dir = "gs://tf_records_bucket/tf_records/Untitled Folder/logs/"  # Specify the directory to save logs
    tensorboard_callback = TensorBoard(log_dir=log_dir)

    output_of_image = "gs://tf_records_bucket/tf_records/Untitled Folder/output"
    
    # Combine all callbacks
    all_callbacks = [
        keras.callbacks.ModelCheckpoint(model_path + model_name, save_best_only=False),
        tensorboard_callback,
        custom_metrics_callback,
        custom_metrics_csv_logger,  # Add the custom_metrics_csv_logger here
        # early_stopping,
        PredictSegmentationCallback(test_image_path, output_of_image)
    ]

    model_history = model.fit(
        train_dataset,
        epochs=3,
        callbacks=all_callbacks,
        batch_size=32,
        validation_data=val_dataset,
    )

    if early_stopping.stopped_epoch > 0:
        print(f"Training stopped at epoch {early_stopping.stopped_epoch} due to early stopping.")
    else:
        print("Training completed all epochs.")
    # Save the model after training
    # model.save(model_path + model_name)
    # print("Model saved locally")

    upload_blob(gcs_path)
    print("Uploaded to cloud storage successfully")



if __name__ == "__main__":
    input_params = {
        "input_directory": "gs://tf_records_bucket/tf_records/Untitled Folder/",  # make sure / is there at the end,
        "bucket_name": "tf_records_bucket",
        "threshold_percentage": 99.9,
        "image_names" : [],
        "image_channels": 8,  # 8 bands images as input
        "label_channels": 1,
        "patch_height": 8,
        "patch_width": 8,
        "batch_size": 32,
        "num_classes": 23,
        "model_path": "trained_model/",
    }  # makesure there is a slash at the end of the path  # Choose an appropriate batch size

    train(**input_params)







# ! tensorboard --logdir gs://tf_records_bucket/tf_records/Untitled\ Folder/logs/ 












Loaded model from checkpoint
TFRecords List: ['gs://tf_records_bucket/tf_records/Untitled Folder/area2_0123_2023_8bands_tfrecord.tfrecord', 'gs://tf_records_bucket/tf_records/Untitled Folder/area2_0617_2023_8bands_tfrecord.tfrecord']


2024-03-09 07:53:30.224563: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2]
	 [[{{node Placeholder/_0}}]]


Dataset: {'image': SparseTensor(indices=tf.Tensor(
[[        0]
 [        1]
 [        2]
 ...
 [135436813]
 [135436814]
 [135436815]], shape=(135436816, 1), dtype=int64), values=tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(135436816,), dtype=float32), dense_shape=tf.Tensor([135436816], shape=(1,), dtype=int64)), 'image_shape': SparseTensor(indices=tf.Tensor(
[[0]
 [1]
 [2]], shape=(3, 1), dtype=int64), values=tf.Tensor([3694 4583    8], shape=(3,), dtype=int64), dense_shape=tf.Tensor([3], shape=(1,), dtype=int64)), 'label': SparseTensor(indices=tf.Tensor(
[[       0]
 [       1]
 [       2]
 ...
 [16929599]
 [16929600]
 [16929601]], shape=(16929602, 1), dtype=int64), values=tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(16929602,), dtype=float32), dense_shape=tf.Tensor([16929602], shape=(1,), dtype=int64)), 'label_shape': SparseTensor(indices=tf.Tensor(
[[0]
 [1]
 [2]], shape=(3, 1), dtype=int64), values=tf.Tensor([3694 4583    1], shape=(3,), dtype=int64), dense_shape=tf.Tensor([3], shape=(1

TypeError: in user code:

    File "/tmp/ipykernel_1/3359277659.py", line 345, in sampling  *
        num_zeros = tf.reduce_sum(tf.cast(tf.equal(label_image, tf.constant(0, dtype=tf.float32)), tf.float32), axis=[2, 3, 4])

    TypeError: Expected float32, but got (3696, 4584, 1) of type 'TensorShape'.


In [ ]:
# imports================

import os
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from google.cloud import storage
from pipeline_scripts.model_file import get_model

# Necessary Functions------------------------------------------

feature_description = {
    "image": tf.io.VarLenFeature(tf.float32),
    "image_shape": tf.io.VarLenFeature(tf.int64),
    "label": tf.io.VarLenFeature(tf.float32),
    "label_shape": tf.io.VarLenFeature(tf.int64),
}


def parse(serialized_examples):
    return tf.io.parse_example(serialized_examples, feature_description)


def create_dataset(input_directory, image_names=[]):
    if len(image_names)==0:
        tfrecord_files = [
        f"{input_directory}{file}"
        for file in tf.io.gfile.listdir(input_directory)
        if file.endswith(".tfrecord")
    ]
    else:
        names = [name.split(".")[0]+"_tfrecord" for name in image_names]
        print(names)
        tfrecord_files = [
            f"{input_directory}{file}"
            for file in tf.io.gfile.listdir(input_directory)
            if file.endswith(".tfrecord") and os.path.basename(file).rsplit('.', 1)[0] in names
        ]
    dataset = tf.data.TFRecordDataset(tfrecord_files)
    dataset = dataset.map(parse, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset





@tf.function
def bandwise_normalize(input_tensor, epsilon=1e-8):
    input_tensor = tf.cast(input_tensor, tf.float32)
    min_val = tf.reduce_min(input_tensor, axis=2, keepdims=True)
    max_val = tf.reduce_max(input_tensor, axis=2, keepdims=True)
    denom = max_val - min_val
    denom = tf.where(tf.abs(denom) < epsilon, epsilon, denom)
    normalized_tensor = (input_tensor - min_val) / denom
    return normalized_tensor

@tf.function
def pad_to_multiple(image, TILE_HT, TILE_WD):
    target_height = tf.cast(tf.math.ceil(tf.shape(image)[0] / TILE_HT) * TILE_HT, tf.int32)
    target_width = tf.cast(tf.math.ceil(tf.shape(image)[1] / TILE_WD) * TILE_WD, tf.int32)
    padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
    return padded_image

@tf.function
def tile_image(fullimg, CHANNELS, TILE_HT, TILE_WD):
    fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
    images = tf.expand_dims(fullimg, axis=0)
    tiles = tf.image.extract_patches(
        images=images,
        sizes=[1, TILE_HT, TILE_WD, 1],
        strides=[1, TILE_HT, TILE_WD, 1],
        rates=[1, 1, 1, 1],
        padding="VALID",
    )
    tiles = tf.squeeze(tiles, axis=0)
    # Cast elements to a supported data type
    tiles = tf.cast(tiles, tf.float32)
    
    # Reshape without specifying batch size dimensions
    tiles_shape = tf.shape(tiles)
    tiles = tf.reshape(tiles, [tiles_shape[0], tiles_shape[1], TILE_HT, TILE_WD, CHANNELS])
    return tiles

@tf.function
def sampling(label_image, threshold_percentage=99.9):
    num_zeros = tf.reduce_sum(tf.cast(tf.equal(label_image, 0), tf.float32), axis=[2, 3, 4])
    total_elements = tf.cast(tf.reduce_prod(tf.shape(label_image)[2:]), tf.float32)
    percentage_zeros = (num_zeros / total_elements) * 100.0
    boolean_mask = percentage_zeros <= threshold_percentage
    return boolean_mask, tf.cast(percentage_zeros >= threshold_percentage, tf.int32)

@tf.function
def one_hot_encoding(label_tensor):
    float_labels = tf.squeeze(label_tensor, axis=-1)
    num_classes = tf.cast(tf.reduce_max(float_labels) + 1, tf.int32)
    one_hot_encoded_images = tf.one_hot(tf.dtypes.cast(float_labels, tf.int32), depth=num_classes)
    return one_hot_encoded_images

# Optimized parsing function using tf.data.Dataset
def parsing(dataset, patch_height, patch_width, threshold_percentage, image_channels, label_channels):
    def parse_example(parsed_example):
        image_shape = tf.sparse.to_dense(parsed_example["image_shape"])
        image = tf.reshape(tf.sparse.to_dense(parsed_example["image"]), image_shape)
        label_shape = tf.sparse.to_dense(parsed_example["label_shape"])
        label = tf.reshape(tf.sparse.to_dense(parsed_example["label"]), label_shape)
        print(image_shape)
        image = bandwise_normalize(image)
        image_patches = tile_image(image, image_channels, patch_height, patch_width)
        label_patches = tile_image(label, label_channels, patch_height, patch_width)
        
        sampled_mask, _ = sampling(label_patches, threshold_percentage)
        sampled_image_patches = tf.boolean_mask(image_patches, sampled_mask)
        sampled_label_patches = tf.boolean_mask(label_patches, sampled_mask)
        
        sampled_label_patches = one_hot_encoding(sampled_label_patches)
        
        return sampled_image_patches, sampled_label_patches

    # Apply the parse_example function to each element in the dataset
    parsed_dataset = dataset.map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    
    image_patch_tensors_list = []
    label_patch_tensors_list = []

    for image_patches, label_patches in parsed_dataset:

        # Append to lists
        image_patch_tensors_list.append(image_patches)
        label_patch_tensors_list.append(label_patches)
        
    return image_patch_tensors_list, label_patch_tensors_list





def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client(project="gislogics")
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print("File {} uploaded to {}.".format(source_file_name, destination_blob_name))

    
def train_test_datasets(
    input_directory,
    patch_height,
    patch_width,
    image_channels,
    label_channels,
    threshold_percentage,
    batch_size,
    image_names
):
    dataset = create_dataset(input_directory, image_names)
    image_patch_tensors_list, label_patch_tensors_list = parsing(
        dataset=dataset,
        patch_height=patch_height,
        patch_width=patch_width,
        image_channels=image_channels,
        label_channels=label_channels,
        threshold_percentage=threshold_percentage,
    )

    # Combine images and labels from different pairs
    combined_images = tf.concat(image_patch_tensors_list, axis=0)
    combined_labels = tf.concat(label_patch_tensors_list, axis=0)

    # Shuffle the combined data
    combined_dataset = tf.data.Dataset.from_tensor_slices(
        (combined_images, combined_labels)
    )
    combined_dataset = combined_dataset.shuffle(buffer_size=combined_images.shape[0])

    # Split the combined dataset into training and validation sets
    train_size = int(0.8 * combined_images.shape[0])
    train_dataset = combined_dataset.take(train_size)
    val_dataset = combined_dataset.skip(train_size)

    # Batch the data using TensorFlow's Dataset API
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)

    return train_dataset, val_dataset

# Modeling--------------------------------------------------


def train(**kwargs):
    input_directory = kwargs.get("input_directory")
    threshold_percentage = kwargs.get("threshold_percentage")
    image_channels = kwargs.get("image_channels")
    label_channels = kwargs.get("label_channels")
    patch_height = kwargs.get("patch_height")
    patch_width = kwargs.get("patch_width")
    batch_size = kwargs.get("batch_size")
    num_classes = kwargs.get("num_classes")
    model_path = kwargs.get("model_path")
    bucket_name = kwargs.get("bucket_name")
    image_names = kwargs.get("image_names")
    img_size = (patch_height, patch_width)
    model_name = f"patch_h{patch_height}_w{patch_width}_batch_{batch_size}_on_0609_2023_composite.hdf5"

    # get the train and test datasets
    train_dataset, val_dataset = train_test_datasets(
        input_directory,
        patch_height,
        patch_width,
        image_channels,
        label_channels,
        threshold_percentage,
        batch_size,
        image_names
    )
    print("Train and Valid datasets are created")

    # create img_size
    model = get_model(
        img_size=img_size, num_classes=num_classes, num_bands=image_channels
    )
    model.compile(optimizer="adam", loss="categorical_crossentropy")
    callbacks = [
        keras.callbacks.ModelCheckpoint(model_path + model_name, save_best_only=False),
        keras.callbacks.CSVLogger(input_directory + "logs/" + f"training_logs_{model_name}_single_file.csv"),  # Add CSVLogger
    ]
    model_history = model.fit(
        train_dataset,
        epochs=3,
        callbacks=callbacks,
        batch_size=32,
        validation_data=val_dataset,
    )
    model.save(model_path + model_name)
    print("model saved locally")

    upload_blob(bucket_name, model_path + model_name, "model/" + model_name)
    print("uploaded to cloud storage successfully")


if __name__ == "__main__":
    input_params = {
        "input_directory": "gs://tf_records_bucket/tf_records/Untitled Folder/",  # make sure / is there at the end,
        "bucket_name": "tf_records_bucket",
        "image_names": [],
        "threshold_percentage": 99.9,
        "image_channels": 8,  # 8 bands images as input
        "label_channels": 1,
        "patch_height": 8,
        "patch_width": 8,
        "batch_size": 32,
        "num_classes": 23,
        "model_path": "trained_model/",
    }  # makesure there is a slash at the end of the path  # Choose an appropriate batch size

    train(**input_params)


2024-03-09 04:58:26.948930: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: label_shape.  Can't parse serialized Example.
2024-03-09 04:58:26.953933: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: Key: label_shape.  Can't parse serialized Example.
	 [[{{node ParseExample/ParseExampleV2}}]]


In [1]:

!pip install rasterio
!pip install geopandas
!pip install Pillow














import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import time
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from shapely.geometry import mapping
from datetime import datetime


from tensorflow.keras.callbacks import Callback
from google.cloud import storage
import os
import csv
from PIL import Image
import io
import json


# policy = tf.keras.mixed_precision.Policy('mixed_float16')
# tf.keras.mixed_precision.set_global_policy(policy)


# metrics_results = compute_metrics(new_ground_truth, new_predict)


# metrics_results

from matplotlib.colors import ListedColormap
class_colors = {
        1: ( 5, 5, 230),
        2: (190, 60, 15),
        3: (65, 240, 125),
        4: (105, 200, 95),
        5: ( 30, 115, 10),
        6: ( 255, 196, 34),
        7: (110, 85, 5),
        8: ( 235, 235, 220),
        9: (120, 216, 47),
        10: ( 84, 142, 128),
        11: ( 84, 142, 128),
        12: ( 84, 142, 128),
        13: ( 50, 255, 215),
        14: ( 50, 255, 215),
        15: ( 50, 255, 215),
        16: ( 193, 255, 0),
        17: ( 105, 200, 95),
        18: (105, 200, 95),
        19: ( 105, 200, 95),
        20: (193, 255, 0),
        21: ( 255, 50, 185),
        22: (255, 255, 255),
}

# Create a colormap using the class-color mapping
colors = [class_colors[i] for i in range(1, 23)]
normalized_colors_array = np.array([tuple(np.array(v) / 255.0) for v in class_colors.values()])

cmap_image = ListedColormap(normalized_colors_array)



class CustomMetricsCSVLogger(Callback):
    def __init__(self, filename, separator=',', append=True):
        super(CustomMetricsCSVLogger, self).__init__()
        self.filename = filename
        self.separator = separator
        self.append = append
        self.keys = None
        self.append_header = True
        self.max_epoch = 0  # Track the highest epoch number encountered
    def on_epoch_begin(self, epoch, logs=None):
        # Initialize min and max class-wise IOU at the beginning of each epoch
        self.min_class_wise_iou = 100
        self.max_class_wise_iou = 0

        # Check if the file exists in Cloud Storage, if not, create it
        if not self.file_exists():
            self.create_file()

    def file_exists(self):
        # Check if the file exists in Cloud Storage
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        return blob.exists()

    def create_file(self):
        # Create the file in Cloud Storage and write the header
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Write the header to the blob
        header = 'epoch,loss,val_loss,class_wise_iou,class_wise_dice_score,class_wise_accuracy,class_wise_precision,class_wise_recall,mean_iou,min_class_wise_iou,max_class_wise_iou\n'
        blob.upload_from_string(header)

    def parse_gcs_path(self, gcs_path):
        # Parse the Google Cloud Storage path to extract bucket name and blob name
        parts = gcs_path.replace('gs://', '').split('/')
        return parts[0], '/'.join(parts[1:])

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if self.keys is None:
            self.keys = sorted(logs.keys())

        # Extract class-wise IOU
        class_wise_iou = logs.get('class_wise_iou', 0.0)

        # Update min and max class-wise IOU
        self.min_class_wise_iou = min(self.min_class_wise_iou, class_wise_iou)
        self.max_class_wise_iou = max(self.max_class_wise_iou, class_wise_iou)

        # Append the row to the file in Cloud Storage
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Download existing content
        existing_content = blob.download_as_text() if blob.exists() else ""

        # Extract metrics values from logs
        metrics_values = [str(logs[key]) for key in ['loss', 'val_loss', 'class_wise_iou', 'class_wise_dice_score',
                                                    'class_wise_accuracy', 'class_wise_precision', 'class_wise_recall', 'mean_iou']]

        # Check if metrics for the current epoch already exist
        epoch_exists = any(f"{epoch}," in line for line in existing_content.split('\n'))

        # If the file is empty or epoch entry doesn't exist, append the metrics
        if not existing_content or not epoch_exists:
            updated_content = existing_content + f"{epoch},{','.join(metrics_values)},{self.min_class_wise_iou},{self.max_class_wise_iou}\n"
        else:
            # Get the maximum epoch number in the existing content
            max_existing_epoch = max(
                int(line.split(',')[0]) for line in existing_content.split('\n') if line.strip() and not line.startswith('epoch')
            )

            # Increment the epoch for the new entries
            updated_content = existing_content + f"{max_existing_epoch + 1},{','.join(metrics_values)},{self.min_class_wise_iou},{self.max_class_wise_iou}\n"

        # Update the highest epoch number
        self.max_epoch = max(self.max_epoch, epoch)

        # Upload updated content
        blob.upload_from_string(updated_content)





from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import os

class PredictSegmentationCallback(Callback):
    def __init__(self, test_image_path, output_save_path):
        super(PredictSegmentationCallback, self).__init__()
        self.test_image_path = test_image_path
        self.output_save_path = output_save_path
        self.last_predicted_array = None

    def parse_gcs_path(self, gcs_path):
        # Parse the Google Cloud Storage path to extract bucket name and blob name
        parts = gcs_path.replace('gs://', '').split('/')
        return parts[0], '/'.join(parts[1:])

    # def on_epoch_end(self, epoch, logs=None):
    #     # Call your prediction function
    #     self.last_predicted_array = prediction_function_img(self.test_image_path)
    #     normalized_array_image = self.last_predicted_array / (np.max(self.last_predicted_array) + 1e-10)
    #     # Display the predicted array with the specified colormap
    #     # plt.figure(figsize=(8, 4))
    #     # plt.imshow(normalized_array_image, cmap=cmap_image)  # Adjust the colormap as needed
    #     # plt.title(f'Predicted Array - Epoch {epoch}')
    #     # plt.show()

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 20 == 0:
            # Save the predicted array every 20 epochs
            # Convert NumPy array to PIL Image
            self.last_predicted_array = prediction_function_img(self.test_image_path)
            pil_image = Image.fromarray((self.last_predicted_array * 255).astype(np.uint8))

            # Apply the colormap to the PIL Image
            pil_image_colored = pil_image.convert('P', palette=Image.ADAPTIVE, colors=len(class_colors))
            pil_image_colored.putpalette(np.array(normalized_colors_array * 255, dtype=np.uint8).flatten())

            # Extract image name from the test_image_path for Google Cloud Storage
            image_name = os.path.basename(self.test_image_path)

            # Create the full GCS path for saving the predicted array as an image
            gcs_content_save_path = f'{self.output_save_path}/output_epoch_{epoch}_{image_name}.png'

            # Upload the image directly to GCS
            storage_client = storage.Client()
            bucket_name, blob_name = self.parse_gcs_path(gcs_content_save_path)
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            # Save the PIL Image to a BytesIO object
            image_io = io.BytesIO()
            pil_image_colored.save(image_io, format='PNG')

            image_io.seek(0)

            # Upload the image content to GCS
            blob.upload_from_file(image_io, content_type='image/png')  # Set the correct content type

            print(f'Predicted array saved at: {gcs_content_save_path}')
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
# imports================
# imports================

import os
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from google.cloud import storage
from pipeline_scripts.model_file import get_model

# Necessary Functions------------------------------------------

feature_description = {
    "image": tf.io.VarLenFeature(tf.float32),
    "image_shape": tf.io.VarLenFeature(tf.int64),
    "label": tf.io.VarLenFeature(tf.float32),
    "label_shape": tf.io.VarLenFeature(tf.int64),
}


def parse(serialized_examples):
    return tf.io.parse_example(serialized_examples, feature_description)


def create_dataset(input_directory, image_names=[]):
    if len(image_names)==0:
        tfrecord_files = [
        f"{input_directory}{file}"
        for file in tf.io.gfile.listdir(input_directory)
        if file.endswith(".tfrecord")
    ]
    else:
        names = [name.split(".")[0]+"_tfrecord" for name in image_names]
        print(names)
        tfrecord_files = [
            f"{input_directory}{file}"
            for file in tf.io.gfile.listdir(input_directory)
            if file.endswith(".tfrecord") and os.path.basename(file).rsplit('.', 1)[0] in names
        ]
    # print(f"TFRecords List: {tfrecord_files}")
    dataset = tf.data.TFRecordDataset(tfrecord_files).map(parse
                                                         ) # , num_parallel_reads=tf.data.AUTOTUNE
    # dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE) 
    return dataset



# # pre-processing functions
# def bandwise_normalize(input_tensor, epsilon=1e-8):
#     input_tensor = tf.cast(input_tensor, tf.float32)
#     min_val = tf.reduce_min(input_tensor, axis=2, keepdims=True)
#     max_val = tf.reduce_max(input_tensor, axis=2, keepdims=True)
#     denom = max_val - min_val
#     denom = tf.where(tf.abs(denom) < epsilon, epsilon, denom)
#     normalized_tensor = (input_tensor - min_val) / denom
#     return normalized_tensor



# def pad_to_multiple(image, TILE_HT, TILE_WD):
#     target_height = tf.cast(tf.math.ceil(tf.shape(image)[0] / TILE_HT) * TILE_HT, tf.int32)
#     target_width = tf.cast(tf.math.ceil(tf.shape(image)[1] / TILE_WD) * TILE_WD, tf.int32)
#     padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
#     return padded_image



# def tile_image(fullimg, CHANNELS, TILE_HT, TILE_WD):
#     fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
#     images = tf.expand_dims(fullimg, axis=0)
#     tiles = tf.image.extract_patches(
#         images=images,
#         sizes=[1, TILE_HT, TILE_WD, 1],
#         strides=[1, TILE_HT, TILE_WD, 1],
#         rates=[1, 1, 1, 1],
#         padding="VALID",
#     )

#     tiles = tf.squeeze(tiles, axis=0)
#     nrows = tiles.shape[0]
#     ncols = tiles.shape[1]
#     tiles = tf.reshape(tiles, [nrows, ncols, TILE_HT, TILE_WD, CHANNELS])
#     return tiles


# def sampling(label_image, threshold_percentage=99.9):
#     num_zeros = tf.reduce_sum(
#         tf.cast(tf.equal(label_image, 0), tf.float32), axis=[2, 3, 4]
#     )
#     total_elements = tf.cast(tf.reduce_prod(tf.shape(label_image)[2:]), tf.float32)
#     percentage_zeros = (num_zeros / total_elements) * 100.0
#     boolean_mask = percentage_zeros <= threshold_percentage
#     sampled_tensor = tf.cast(percentage_zeros >= threshold_percentage, tf.int32)
#     return boolean_mask, sampled_tensor



# def one_hot_encoding(label_tensor):
#     float_labels = tf.squeeze(label_tensor, axis=-1)
#     num_classes = tf.cast(tf.reduce_max(float_labels) + 1, tf.int32)
#     one_hot_encoded_images = tf.one_hot(tf.dtypes.cast(float_labels, tf.int32), depth=num_classes)
#     return one_hot_encoded_images


# # Optimized parsing function using tf.data.Dataset
# def parsing(dataset, patch_height, patch_width, threshold_percentage, image_channels, label_channels):
#     def parse_example(parsed_example):
#         image_shape = tf.sparse.to_dense(parsed_example["image_shape"])
#         # print("parsed_example", parsed_example)
#         # print('parsed_example["image_shape"]', parsed_example["image_shape"])
#         # print("image shape in parse example", image_shape)
#         image = tf.reshape(tf.sparse.to_dense(parsed_example["image"]), image_shape)
#         label_shape = tf.sparse.to_dense(parsed_example["label_shape"])
#         label = tf.reshape(tf.sparse.to_dense(parsed_example["label"]), label_shape)
        
#         image = bandwise_normalize(image)
#         print(f"Bandwise Norm Image Shape: {image.shape}")
#         image_patches = tile_image(image, image_channels, patch_height, patch_width)
#         label_patches = tile_image(label, label_channels, patch_height, patch_width)
        
#         sampled_mask, _ = sampling(label_patches, threshold_percentage)
#         sampled_image_patches = tf.boolean_mask(image_patches, sampled_mask)
#         sampled_label_patches = tf.boolean_mask(label_patches, sampled_mask)
        
#         sampled_label_patches = one_hot_encoding(sampled_label_patches)
        
#         return sampled_image_patches, sampled_label_patches
    
#     image_patch_tensors_list = []
#     label_patch_tensors_list = []
#     for parsed_data in dataset:
#         sampled_image_patches, sampled_label_patches = parse_example(parsed_data)
#         image_patch_tensors_list.append(sampled_image_patches)
#         label_patch_tensors_list.append(sampled_label_patches)
#     return image_patch_tensors_list, label_patch_tensors_list

def bandwise_normalize(input_tensor, epsilon=1e-8):
    # Convert the input_tensor to a float32 type
    input_tensor = tf.cast(input_tensor, tf.float32)

    # Calculate the minimum and maximum values along the channel axis
    min_val = tf.reduce_min(input_tensor, axis=2, keepdims=True)
    max_val = tf.reduce_max(input_tensor, axis=2, keepdims=True)

    # Check for potential numerical instability
    denom = max_val - min_val
    denom = tf.where(tf.abs(denom) < epsilon, epsilon, denom)

    # Normalize the tensor band-wise to the range [0, 1]
    normalized_tensor = (input_tensor - min_val) / denom

    return normalized_tensor


def pad_to_multiple(image, TILE_HT, TILE_WD):
    # Get the current dimensions
    height, width, channels = image.shape

    # Calculate the target dimensions
    target_height = tf.cast(tf.math.ceil(height / TILE_HT) * TILE_HT, tf.int32)
    target_width = tf.cast(tf.math.ceil(width / TILE_WD) * TILE_WD, tf.int32)

    # Calculate the amount of padding
    pad_height = target_height - height
    pad_width = target_width - width

    # Pad the image
    padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)

    return padded_image


def tile_image(fullimg, CHANNELS, TILE_HT, TILE_WD):
    fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
    images = tf.expand_dims(fullimg, axis=0)
    tiles = tf.image.extract_patches(
        images=images,
        sizes=[1, TILE_HT, TILE_WD, 1],
        strides=[1, TILE_HT, TILE_WD, 1],
        rates=[1, 1, 1, 1],
        padding="VALID",
    )

    tiles = tf.squeeze(tiles, axis=0)
    nrows = tiles.shape[0]
    ncols = tiles.shape[1]
    tiles = tf.reshape(tiles, [nrows, ncols, TILE_HT, TILE_WD, CHANNELS])
    return tiles


def sampling(label_image, threshold_percentage=99.9):
    num_zeros = tf.reduce_sum(
        tf.cast(tf.equal(label_image, 0), tf.float32), axis=[2, 3, 4]
    )

    # Calculate the total number of elements in each patch
    total_elements = tf.cast(tf.reduce_prod(tf.shape(label_image)[2:]), tf.float32)

    # Calculate the percentage of zeros in each patch
    percentage_zeros = (num_zeros / total_elements) * 100.0

    boolean_mask = percentage_zeros <= threshold_percentage
    # Apply the threshold logic
    sampled_tensor = tf.cast(percentage_zeros >= threshold_percentage, tf.int32)
    return boolean_mask, sampled_tensor


def one_hot_encoding(label_tensor):
    # Assuming your pixel values are float labels
    float_labels = tf.squeeze(
        label_tensor, axis=-1
    )  # Assuming channel dimension is the last one

    # Determine the number of classes dynamically
    num_classes = tf.cast(tf.reduce_max(float_labels) + 1, tf.int32)

    # One-hot encode each image
    one_hot_encoded_images = tf.one_hot(
        tf.dtypes.cast(float_labels, tf.int32), depth=num_classes
    )

    # Print the shape of the resulting tensor and the number of classes
    # print("Shape of one-hot encoded images:", one_hot_encoded_images.shape)
    # print("Number of classes:", num_classes)

    return one_hot_encoded_images

def parsing(
    dataset,
    patch_height,
    patch_width,
    threshold_percentage,
    image_channels,
    label_channels,
):
    image_patch_tensors_list = []
    label_patch_tensors_list = []

    for parsed_example in dataset:
        image_shape = tf.sparse.to_dense(parsed_example["image_shape"])
        image = tf.reshape(tf.sparse.to_dense(parsed_example["image"]), image_shape)
        label_shape = tf.sparse.to_dense(parsed_example["label_shape"])
        label = tf.reshape(tf.sparse.to_dense(parsed_example["label"]), label_shape)

        # image normalization
        image = bandwise_normalize(image)

        # image and label patching
        image_patches = tile_image(image, image_channels, patch_height, patch_width)
        label_patches = tile_image(label, label_channels, patch_height, patch_width)

        # sampling
        sampled_mask, sampled_tensor = sampling(label_patches, threshold_percentage)
        sampled_image_patches = tf.boolean_mask(image_patches, sampled_mask)
        sampled_label_patches = tf.boolean_mask(label_patches, sampled_mask)

        # one-hot encoding
        sampled_label_patches = one_hot_encoding(sampled_label_patches)

        # save them in the list
        image_patch_tensors_list.append(sampled_image_patches)
        label_patch_tensors_list.append(sampled_label_patches)

    return image_patch_tensors_list, label_patch_tensors_list


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client(project="gislogics")
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print("File {} uploaded to {}.".format(source_file_name, destination_blob_name))

    
def train_test_datasets(
    input_directory,
    patch_height,
    patch_width,
    image_channels,
    label_channels,
    threshold_percentage,
    batch_size,
    image_names
):
    dataset = create_dataset(input_directory, image_names)
    image_patch_tensors_list, label_patch_tensors_list = parsing(
        dataset=dataset,
        patch_height=patch_height,
        patch_width=patch_width,
        image_channels=image_channels,
        label_channels=label_channels,
        threshold_percentage=threshold_percentage,
    )
    
    combined_images = tf.concat(image_patch_tensors_list, axis=0)
    combined_labels = tf.concat(label_patch_tensors_list, axis=0)

    # Shuffle the combined data
    combined_dataset = tf.data.Dataset.from_tensor_slices(
        (combined_images, combined_labels)
    )
    combined_dataset = combined_dataset.shuffle(buffer_size=combined_images.shape[0])

    # Split the combined dataset into training and validation sets
    train_size = int(0.8 * combined_images.shape[0])
    train_dataset = combined_dataset.take(train_size)
    val_dataset = combined_dataset.skip(train_size)

    # Batch the data using TensorFlow's Dataset API
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)

    return train_dataset, val_dataset
    

def load_config(file_path):
    with open(file_path, 'r') as f:
        config = json.load(f)
    return config








# Modeling--------------------------------------------------
config = load_config('./pipeline_scripts/config.json')
from google.cloud import storage
from tensorflow.keras.models import save_model, load_model
from tensorflow.python.lib.io import file_io
# Convert img_size to a tuple
config["img_size"] = tuple(config["img_size"])

# Now you can access the config values normally
model_path = config.get("model_path")
model_name = config.get("model_name")
test_image_path = config.get("test_image_path")
img_size = config.get("img_size")
num_bands = config.get("num_bands")
num_classes = config.get("num_classes")
gcs_path = config.get("google_storage_path")
gcs_tfrecords = config.get("gcs_tfrecords")


# Load the model if a checkpoint exists
# Load the model if a checkpoint exists
import gcsfs

fs = gcsfs.GCSFileSystem(project='gislogics')  # Set up GCS connection

if tf.io.gfile.exists(model_path):  # Accurately check for file existence in GCS
    # Load existing model
    model = load_model(model_path)  # Handle GCS authentication here if needed
    print("Loaded model from checkpoint")
else:
    # If no checkpoint exists, create a new model
    model = get_model(
        img_size=img_size, num_classes=num_classes, num_bands=num_bands
    )
    model.compile(optimizer="adam", loss="categorical_crossentropy")
    print("Created a new model")

    # # Save the new model to the model_path in GCS
    save_model(model, model_path)
    print("Saved new model to:", model_path)

    
    
    
    
# Continue training
from pipeline_scripts.eval import *
from pipeline_scripts.prediction import *

def train(**kwargs):
    input_directory = kwargs.get("input_directory")
    threshold_percentage = kwargs.get("threshold_percentage")
    image_channels = kwargs.get("image_channels")
    label_channels = kwargs.get("label_channels")
    patch_height = kwargs.get("patch_height")
    patch_width = kwargs.get("patch_width")
    batch_size = kwargs.get("batch_size")
    num_classes = kwargs.get("num_classes")  
    bucket_name = kwargs.get("bucket_name")

    image_names = kwargs.get("image_names")
    # img_size = (patch_height, patch_width)
    
    

    # get the train and test datasets
    train_dataset, val_dataset = train_test_datasets(
        gcs_tfrecords,
        patch_height,
        patch_width,
        image_channels,
        label_channels,
        threshold_percentage,
        batch_size,
        image_names
    )
    print("Train and Valid datasets are created")

    # create img_size
    model = get_model(
        img_size= img_size, 
        num_classes=num_classes, 
        num_bands=image_channels
    )
    
    
    # compilation of model, with custom metric
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy", 
        metrics=[np.mean(compute_metrics(new_ground_truth, new_predict)[0])]
    )

    
    # Early stopping after 5 epochs 
    early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,  
    restore_best_weights=True,  
    verbose=1  
    )
    
    # Agroforestry Class
    Agroforestry = 18
    
    # including custom metrics in callbacks
    custom_metrics_callback = keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs: logs.update({
        "class_wise_iou": compute_metrics(new_ground_truth, new_predict)[0][Agroforestry],
        "class_wise_dice_score": compute_metrics(new_ground_truth, new_predict)[1][Agroforestry],
        "class_wise_accuracy": compute_metrics(new_ground_truth, new_predict)[2][Agroforestry],
        "class_wise_precision": compute_metrics(new_ground_truth, new_predict)[3][Agroforestry],
        "class_wise_recall": compute_metrics(new_ground_truth, new_predict)[4][Agroforestry],
        "mean_iou": compute_metrics(new_ground_truth, new_predict)[5],
        "min_class_wise_iou": np.min(metrics_results[0]),
        "max_class_wise_iou": np.max(metrics_results[0]),
        "epoch": epoch,
        "loss": logs["loss"],
        "val_loss": logs["val_loss"]}))

    
    # callbacks and logging
    csv_logger = keras.callbacks.CSVLogger(
    input_directory + "logs/" + f"training_logs_{model_name}.csv",
    append=True
    )

    custom_metrics_csv_logger = CustomMetricsCSVLogger(
        input_params["input_directory"] + "logs/" + f"training_logs_{model_name}.csv",
        append=True
    )
    
    
    log_dir = "gs://tf_records_bucket/tf_records/Untitled Folder/logs/"  # Specify the directory to save logs
    tensorboard_callback = TensorBoard(log_dir=log_dir)

    output_of_image = "gs://tf_records_bucket/tf_records/Untitled Folder/output"
    
    # Combine all callbacks
    all_callbacks = [
        keras.callbacks.ModelCheckpoint(model_path + model_name, save_best_only=False),
        tensorboard_callback,
        custom_metrics_callback,
        custom_metrics_csv_logger,  # Add the custom_metrics_csv_logger here
        # early_stopping,
        PredictSegmentationCallback(test_image_path, output_of_image)
    ]

    model_history = model.fit(
        train_dataset,
        epochs=3,
        callbacks=all_callbacks,
        batch_size=32,
        validation_data=val_dataset,
    )

    if early_stopping.stopped_epoch > 0:
        print(f"Training stopped at epoch {early_stopping.stopped_epoch} due to early stopping.")
    else:
        print("Training completed all epochs.")
    # Save the model after training
    # model.save(model_path + model_name)
    # print("Model saved locally")

    upload_blob(gcs_path)
    print("Uploaded to cloud storage successfully")



if __name__ == "__main__":
    input_params = {
        "input_directory": "gs://tf_records_bucket/tf_records/Untitled Folder/",  # make sure / is there at the end,
        "bucket_name": "tf_records_bucket",
        "threshold_percentage": 99.9,
        "image_names" : [],
        "image_channels": 8,  # 8 bands images as input
        "label_channels": 1,
        "patch_height": 8,
        "patch_width": 8,
        "batch_size": 32,
        "num_classes": 23,
        "model_path": "trained_model/",
    }  # makesure there is a slash at the end of the path  # Choose an appropriate batch size

    train(**input_params)



  Using cached rasterio-1.3.9-cp310-cp310-manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl.metadata (3.4 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
Using cached rasterio-1.3.9-cp310-cp310-manylinux2014_x86_64.whl (20.6 MB)
Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
Using cached affine-2.4.0-py3-none-any.whl (15 kB)
Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached geopandas-0.14.3-py3-none-any.whl.metadata (1.5 kB)
  Using cached fiona-1.9.6-cp310-cp310-manylinux2014_x86_64.whl.metadata (50 kB)
  Using cached pyproj-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
Using cached geopandas-0.14.3-py3-none-any.whl (1.1 MB)
Using cached fiona-1.9.6-cp310-cp310-manylinux2014_x86

2024-03-09 13:06:34.958866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 13:06:45.055929: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load li

Loaded model from checkpoint
prediction.clip_tiff, tiff gs://tf_records_bucket/labels/area2_0530_2022_8bands.tif
(3694, 4560, 8)
(3694, 4560, 8)
dims (3696, 4560, 8)
129/129 [==============================] - 9s 68ms/step
Time taken for predictions: 10.997666597366333 seconds
segmentation_patches_reshaped.shape (462, 570, 8, 8)
row_indices_patch (462,)
col_indices_patch (570,)
stitched_array (3696, 4560)
Time taken including stitching: 12.731143236160278 seconds
prediction.clip_tiff, tiff gs://tf_records_bucket/labels/continuous_label_raster.tif


2024-03-09 13:07:54.338421: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2]
	 [[{{node Placeholder/_0}}]]
2024-03-09 13:07:54.338725: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2]
	 [[{{node Placeholder/_0}}]]


(3694, 4583, 8)
(3694, 4583, 8)
(3694, 4583, 1)
(3694, 4583, 1)


InvalidArgumentError: {{function_node __wrapped__Pack_N_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [462,573,8,8,1] != values[1].shape = [264726,8,8,1] [Op:Pack]

## Testing on only train part 

In [6]:
# imports================

import os
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from google.cloud import storage
from pipeline_scripts.model_file import get_model
import json
from datetime import datetime
# Necessary Functions------------------------------------------




feature_description = {
    "image": tf.io.VarLenFeature(tf.float32),
    "image_shape": tf.io.VarLenFeature(tf.int64),
    "label": tf.io.VarLenFeature(tf.float32),
    "label_shape": tf.io.VarLenFeature(tf.int64),
}



def parse(serialized_examples):
    return tf.io.parse_example(serialized_examples, feature_description)



def create_dataset(input_directory, image_names=[]):
    if len(image_names)==0:
        tfrecord_files = [
        f"{input_directory}{file}"
        for file in tf.io.gfile.listdir(input_directory)
        if file.endswith(".tfrecord")
    ]
    else:
        names = [name.split(".")[0]+"_tfrecord" for name in image_names]
        print(names)
        tfrecord_files = [
            f"{input_directory}{file}"
            for file in tf.io.gfile.listdir(input_directory)
            if file.endswith(".tfrecord") and os.path.basename(file).rsplit('.', 1)[0] in names
        ]
    dataset = tf.data.TFRecordDataset(tfrecord_files)
    dataset = dataset.map(parse)
    return dataset


# # pre-processing functions
# def bandwise_normalize(input_tensor, epsilon=1e-8):
#     # Convert the input_tensor to a float32 type
#     input_tensor = tf.cast(input_tensor, tf.float32)

#     # Calculate the minimum and maximum values along the channel axis
#     min_val = tf.reduce_min(input_tensor, axis=2, keepdims=True)
#     max_val = tf.reduce_max(input_tensor, axis=2, keepdims=True)

#     # Check for potential numerical instability
#     denom = max_val - min_val
#     denom = tf.where(tf.abs(denom) < epsilon, epsilon, denom)

#     # Normalize the tensor band-wise to the range [0, 1]
#     normalized_tensor = (input_tensor - min_val) / denom

#     return normalized_tensor


# def pad_to_multiple(image, TILE_HT, TILE_WD):
#     # Get the current dimensions
#     height, width, channels = image.shape

#     # Calculate the target dimensions
#     target_height = tf.cast(tf.math.ceil(height / TILE_HT) * TILE_HT, tf.int32)
#     target_width = tf.cast(tf.math.ceil(width / TILE_WD) * TILE_WD, tf.int32)

#     # Calculate the amount of padding
#     pad_height = target_height - height
#     pad_width = target_width - width

#     # Pad the image
#     padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)

#     return padded_image


# def tile_image(fullimg, CHANNELS, TILE_HT, TILE_WD):
#     fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
#     images = tf.expand_dims(fullimg, axis=0)
#     tiles = tf.image.extract_patches(
#         images=images,
#         sizes=[1, TILE_HT, TILE_WD, 1],
#         strides=[1, TILE_HT, TILE_WD, 1],
#         rates=[1, 1, 1, 1],
#         padding="VALID",
#     )

#     tiles = tf.squeeze(tiles, axis=0)
#     nrows = tiles.shape[0]
#     ncols = tiles.shape[1]
#     tiles = tf.reshape(tiles, [nrows, ncols, TILE_HT, TILE_WD, CHANNELS])
#     return tiles


# def sampling(label_image, threshold_percentage=99.9):
#     num_zeros = tf.reduce_sum(
#         tf.cast(tf.equal(label_image, 0), tf.float32), axis=[2, 3, 4]
#     )

#     # Calculate the total number of elements in each patch
#     total_elements = tf.cast(tf.reduce_prod(tf.shape(label_image)[2:]), tf.float32)

#     # Calculate the percentage of zeros in each patch
#     percentage_zeros = (num_zeros / total_elements) * 100.0

#     boolean_mask = percentage_zeros <= threshold_percentage
#     # Apply the threshold logic
#     sampled_tensor = tf.cast(percentage_zeros >= threshold_percentage, tf.int32)
#     return boolean_mask, sampled_tensor


# def one_hot_encoding(label_tensor):
#     # Assuming your pixel values are float labels
#     float_labels = tf.squeeze(
#         label_tensor, axis=-1
#     )  # Assuming channel dimension is the last one

#     # Determine the number of classes dynamically
#     num_classes = tf.cast(tf.reduce_max(float_labels) + 1, tf.int32)

#     # One-hot encode each image
#     one_hot_encoded_images = tf.one_hot(
#         tf.dtypes.cast(float_labels, tf.int32), depth=num_classes
#     )

#     # Print the shape of the resulting tensor and the number of classes
#     # print("Shape of one-hot encoded images:", one_hot_encoded_images.shape)
#     # print("Number of classes:", num_classes)

#     return one_hot_encoded_images

def bandwise_normalize(input_tensor, epsilon=1e-8):
    input_tensor = tf.cast(input_tensor, tf.float32)
    min_val = tf.reduce_min(input_tensor, axis=2, keepdims=True)
    max_val = tf.reduce_max(input_tensor, axis=2, keepdims=True)
    denom = max_val - min_val
    denom = tf.where(tf.abs(denom) < epsilon, epsilon, denom)
    normalized_tensor = (input_tensor - min_val) / denom
    return normalized_tensor



def pad_to_multiple(image, TILE_HT, TILE_WD):
    target_height = tf.cast(tf.math.ceil(tf.shape(image)[0] / TILE_HT) * TILE_HT, tf.int32)
    target_width = tf.cast(tf.math.ceil(tf.shape(image)[1] / TILE_WD) * TILE_WD, tf.int32)
    padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
    return padded_image



def tile_image(fullimg, CHANNELS, TILE_HT, TILE_WD):
    fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
    images = tf.expand_dims(fullimg, axis=0)
    tiles = tf.image.extract_patches(
        images=images,
        sizes=[1, TILE_HT, TILE_WD, 1],
        strides=[1, TILE_HT, TILE_WD, 1],
        rates=[1, 1, 1, 1],
        padding="VALID",
    )

    tiles = tf.squeeze(tiles, axis=0)
    tiles = tf.cast(tiles, tf.float32)
    tiles = tf.reshape(tiles, [tf.shape(tiles)[0], tf.shape(tiles)[1], TILE_HT, TILE_WD, CHANNELS])
    return tiles


def sampling(label_image, threshold_percentage=99.9):
    num_zeros = tf.reduce_sum(
        tf.cast(tf.equal(label_image, 0), tf.float32), axis=[2, 3, 4]
    )
    total_elements = tf.cast(tf.reduce_prod(tf.shape(label_image)[2:]), tf.float32)
    percentage_zeros = (num_zeros / total_elements) * 100.0
    boolean_mask = percentage_zeros <= threshold_percentage
    sampled_tensor = tf.cast(percentage_zeros >= threshold_percentage, tf.int32)
    return boolean_mask, sampled_tensor



def one_hot_encoding(label_tensor):
    float_labels = tf.squeeze(label_tensor, axis=-1)
    num_classes = tf.cast(tf.reduce_max(float_labels) + 1, tf.int32)
    one_hot_encoded_images = tf.one_hot(tf.dtypes.cast(float_labels, tf.int32), depth=num_classes)
    return one_hot_encoded_images



def parsing(
    dataset,
    patch_height,
    patch_width,
    threshold_percentage,
    image_channels,
    label_channels,
):
    image_patch_tensors_list = []
    label_patch_tensors_list = []
    
    
    def _parse(parsed_example):
        image_shape = tf.sparse.to_dense(parsed_example["image_shape"])
        image = tf.reshape(tf.sparse.to_dense(parsed_example["image"]), image_shape)
        label_shape = tf.sparse.to_dense(parsed_example["label_shape"])
        label = tf.reshape(tf.sparse.to_dense(parsed_example["label"]), label_shape)
        
        # Assuming image_channels is known and fixed
        # image.set_shape([None, None, image_channels])
        # label.set_shape([None, None, 1])
        
        # image normalization
        image = bandwise_normalize(image)

        # image and label patching
        image_patches = tile_image(image, image_channels, patch_height, patch_width)
        label_patches = tile_image(label, label_channels, patch_height, patch_width)

        # sampling
        sampled_mask, sampled_tensor = sampling(label_patches, threshold_percentage)
        sampled_image_patches = tf.boolean_mask(image_patches, sampled_mask)
        sampled_label_patches = tf.boolean_mask(label_patches, sampled_mask)

        # one-hot encoding
        sampled_label_patches = one_hot_encoding(sampled_label_patches)

        return sampled_image_patches, sampled_label_patches
    
    parsed_dataset = dataset.map(lambda x: _parse(x))
    for sampled_image_patches, sampled_label_patches in parsed_dataset:
        image_patch_tensors_list.append(sampled_image_patches)
        label_patch_tensors_list.append(sampled_label_patches)

    return image_patch_tensors_list, label_patch_tensors_list



def train_test_datasets(
    input_directory,
    patch_height,
    patch_width,
    image_channels,
    label_channels,
    threshold_percentage,
    batch_size,
    image_names
):
    dataset = create_dataset(input_directory, image_names)
    image_patch_tensors_list, label_patch_tensors_list = parsing(
        dataset=dataset,
        patch_height=patch_height,
        patch_width=patch_width,
        image_channels=image_channels,
        label_channels=label_channels,
        threshold_percentage=threshold_percentage,
    )

    # Combine images and labels from different pairs
    combined_images = tf.concat(image_patch_tensors_list, axis=0)
    combined_labels = tf.concat(label_patch_tensors_list, axis=0)

    # Shuffle the combined data
    combined_dataset = tf.data.Dataset.from_tensor_slices(
        (combined_images, combined_labels)
    )
    combined_dataset = combined_dataset.shuffle(buffer_size=combined_images.shape[0])

    # Split the combined dataset into training and validation sets
    train_size = int(0.8 * combined_images.shape[0])
    train_dataset = combined_dataset.take(train_size)
    val_dataset = combined_dataset.skip(train_size)

    # Batch the data using TensorFlow's Dataset API
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)

    return train_dataset, val_dataset





def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client(project="gislogics")
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print("File {} uploaded to {}.".format(source_file_name, destination_blob_name))


# Modeling--------------------------------------------------


def train(**kwargs):
    input_directory = kwargs.get("input_directory")
    threshold_percentage = kwargs.get("threshold_percentage")
    image_channels = kwargs.get("image_channels")
    label_channels = kwargs.get("label_channels")
    patch_height = kwargs.get("patch_height")
    patch_width = kwargs.get("patch_width")
    batch_size = kwargs.get("batch_size")
    num_classes = kwargs.get("num_classes")
    # model_path = config.get("model_path")
    bucket_name = kwargs.get("bucket_name")
    img_size = (patch_height, patch_width)
    # model_name = config.get("model_name")
    image_names = kwargs.get("image_names")

    # get the train and test datasets
    train_dataset, val_dataset = train_test_datasets(
        input_directory,
        patch_height,
        patch_width,
        image_channels,
        label_channels,
        threshold_percentage,
        batch_size,
        image_names
    )
    print("Train and Valid datasets are created")


if __name__ == "__main__":
    input_params = {
        "input_directory": "gs://tf_records_bucket/tf_records/Untitled Folder/",  # make sure / is there at the end,
        "image_names": [],
        "bucket_name": "tf_records_bucket",
        "threshold_percentage": 99.9,
        "image_channels": 8,  # 8 bands images as input
        "label_channels": 1,
        "patch_height": 8,
        "patch_width": 8,
        "batch_size": 32,
        "num_classes": 23,
        # "model_path": "trained_model/",
    }  # makesure there is a slash at the end of the path  # Choose an appropriate batch size

    train(**input_params)


2024-03-09 16:38:11.647764: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2]
	 [[{{node Placeholder/_0}}]]
2024-03-09 16:38:12.276841: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1780] (One-time warning): Not using XLA:CPU for cluster.

If you want XLA:CPU, do one of the following:

 - set the TF_XLA_FLAGS to include "--tf_xla_cpu_global_jit", or
 - set cpu_global_jit to true on this session's OptimizerOptions, or
 - use experimental_jit_scope, or
 - use tf.function(jit_compile=True).

To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a
proper command-line flag, not via TF_XLA_FLAGS).


Train and Valid datasets are created


## Testing on whole codebase 

In [1]:

!pip install rasterio
!pip install geopandas
!pip install Pillow














import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import time
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from shapely.geometry import mapping
from datetime import datetime


from tensorflow.keras.callbacks import Callback
from google.cloud import storage
import os
import csv
from PIL import Image
import io
import json


# policy = tf.keras.mixed_precision.Policy('mixed_float16')
# tf.keras.mixed_precision.set_global_policy(policy)


# metrics_results = compute_metrics(new_ground_truth, new_predict)


# metrics_results

from matplotlib.colors import ListedColormap
class_colors = {
        1: ( 5, 5, 230),
        2: (190, 60, 15),
        3: (65, 240, 125),
        4: (105, 200, 95),
        5: ( 30, 115, 10),
        6: ( 255, 196, 34),
        7: (110, 85, 5),
        8: ( 235, 235, 220),
        9: (120, 216, 47),
        10: ( 84, 142, 128),
        11: ( 84, 142, 128),
        12: ( 84, 142, 128),
        13: ( 50, 255, 215),
        14: ( 50, 255, 215),
        15: ( 50, 255, 215),
        16: ( 193, 255, 0),
        17: ( 105, 200, 95),
        18: (105, 200, 95),
        19: ( 105, 200, 95),
        20: (193, 255, 0),
        21: ( 255, 50, 185),
        22: (255, 255, 255),
}

# Create a colormap using the class-color mapping
colors = [class_colors[i] for i in range(1, 23)]
normalized_colors_array = np.array([tuple(np.array(v) / 255.0) for v in class_colors.values()])

cmap_image = ListedColormap(normalized_colors_array)



class CustomMetricsCSVLogger(Callback):
    def __init__(self, filename, separator=',', append=True):
        super(CustomMetricsCSVLogger, self).__init__()
        self.filename = filename
        self.separator = separator
        self.append = append
        self.keys = None
        self.append_header = True
        self.max_epoch = 0  # Track the highest epoch number encountered
    def on_epoch_begin(self, epoch, logs=None):
        # Initialize min and max class-wise IOU at the beginning of each epoch
        self.min_class_wise_iou = 100
        self.max_class_wise_iou = 0

        # Check if the file exists in Cloud Storage, if not, create it
        if not self.file_exists():
            self.create_file()

    def file_exists(self):
        # Check if the file exists in Cloud Storage
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        return blob.exists()

    def create_file(self):
        # Create the file in Cloud Storage and write the header
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Write the header to the blob
        header = 'epoch,loss,val_loss,class_wise_iou,class_wise_dice_score,class_wise_accuracy,class_wise_precision,class_wise_recall,mean_iou,min_class_wise_iou,max_class_wise_iou\n'
        blob.upload_from_string(header)

    def parse_gcs_path(self, gcs_path):
        # Parse the Google Cloud Storage path to extract bucket name and blob name
        parts = gcs_path.replace('gs://', '').split('/')
        return parts[0], '/'.join(parts[1:])

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if self.keys is None:
            self.keys = sorted(logs.keys())

        # Extract class-wise IOU
        class_wise_iou = logs.get('class_wise_iou', 0.0)

        # Update min and max class-wise IOU
        self.min_class_wise_iou = min(self.min_class_wise_iou, class_wise_iou)
        self.max_class_wise_iou = max(self.max_class_wise_iou, class_wise_iou)

        # Append the row to the file in Cloud Storage
        storage_client = storage.Client()
        bucket_name, blob_name = self.parse_gcs_path(self.filename)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Download existing content
        existing_content = blob.download_as_text() if blob.exists() else ""

        # Extract metrics values from logs
        metrics_values = [str(logs[key]) for key in ['loss', 'val_loss', 'class_wise_iou', 'class_wise_dice_score',
                                                    'class_wise_accuracy', 'class_wise_precision', 'class_wise_recall', 'mean_iou']]

        # Check if metrics for the current epoch already exist
        epoch_exists = any(f"{epoch}," in line for line in existing_content.split('\n'))

        # If the file is empty or epoch entry doesn't exist, append the metrics
        if not existing_content or not epoch_exists:
            updated_content = existing_content + f"{epoch},{','.join(metrics_values)},{self.min_class_wise_iou},{self.max_class_wise_iou}\n"
        else:
            # Get the maximum epoch number in the existing content
            max_existing_epoch = max(
                int(line.split(',')[0]) for line in existing_content.split('\n') if line.strip() and not line.startswith('epoch')
            )

            # Increment the epoch for the new entries
            updated_content = existing_content + f"{max_existing_epoch + 1},{','.join(metrics_values)},{self.min_class_wise_iou},{self.max_class_wise_iou}\n"

        # Update the highest epoch number
        self.max_epoch = max(self.max_epoch, epoch)

        # Upload updated content
        blob.upload_from_string(updated_content)





from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import os

class PredictSegmentationCallback(Callback):
    def __init__(self, test_image_path, output_save_path):
        super(PredictSegmentationCallback, self).__init__()
        self.test_image_path = test_image_path
        self.output_save_path = output_save_path
        self.last_predicted_array = None

    def parse_gcs_path(self, gcs_path):
        # Parse the Google Cloud Storage path to extract bucket name and blob name
        parts = gcs_path.replace('gs://', '').split('/')
        return parts[0], '/'.join(parts[1:])

    # def on_epoch_end(self, epoch, logs=None):
    #     # Call your prediction function
    #     self.last_predicted_array = prediction_function_img(self.test_image_path)
    #     normalized_array_image = self.last_predicted_array / (np.max(self.last_predicted_array) + 1e-10)
    #     # Display the predicted array with the specified colormap
    #     # plt.figure(figsize=(8, 4))
    #     # plt.imshow(normalized_array_image, cmap=cmap_image)  # Adjust the colormap as needed
    #     # plt.title(f'Predicted Array - Epoch {epoch}')
    #     # plt.show()

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 20 == 0:
            # Save the predicted array every 20 epochs
            # Convert NumPy array to PIL Image
            self.last_predicted_array = prediction_function_img(self.test_image_path)
            pil_image = Image.fromarray((self.last_predicted_array * 255).astype(np.uint8))

            # Apply the colormap to the PIL Image
            pil_image_colored = pil_image.convert('P', palette=Image.ADAPTIVE, colors=len(class_colors))
            pil_image_colored.putpalette(np.array(normalized_colors_array * 255, dtype=np.uint8).flatten())

            # Extract image name from the test_image_path for Google Cloud Storage
            image_name = os.path.basename(self.test_image_path)

            # Create the full GCS path for saving the predicted array as an image
            gcs_content_save_path = f'{self.output_save_path}/output_epoch_{epoch}_{image_name}.png'

            # Upload the image directly to GCS
            storage_client = storage.Client()
            bucket_name, blob_name = self.parse_gcs_path(gcs_content_save_path)
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            # Save the PIL Image to a BytesIO object
            image_io = io.BytesIO()
            pil_image_colored.save(image_io, format='PNG')

            image_io.seek(0)

            # Upload the image content to GCS
            blob.upload_from_file(image_io, content_type='image/png')  # Set the correct content type

            print(f'Predicted array saved at: {gcs_content_save_path}')
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
            
# imports================
# imports================

import os
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from google.cloud import storage
from pipeline_scripts.model_file import get_model

# Necessary Functions------------------------------------------

feature_description = {
    "image": tf.io.VarLenFeature(tf.float32),
    "image_shape": tf.io.VarLenFeature(tf.int64),
    "label": tf.io.VarLenFeature(tf.float32),
    "label_shape": tf.io.VarLenFeature(tf.int64),
}


def parse(serialized_examples):
    return tf.io.parse_example(serialized_examples, feature_description)


def create_dataset(input_directory, image_names=[]):
    if len(image_names)==0:
        tfrecord_files = [
        f"{input_directory}{file}"
        for file in tf.io.gfile.listdir(input_directory)
        if file.endswith(".tfrecord")
    ]
    else:
        names = [name.split(".")[0]+"_tfrecord" for name in image_names]
        print(names)
        tfrecord_files = [
            f"{input_directory}{file}"
            for file in tf.io.gfile.listdir(input_directory)
            if file.endswith(".tfrecord") and os.path.basename(file).rsplit('.', 1)[0] in names
        ]
    # print(f"TFRecords List: {tfrecord_files}")
    dataset = tf.data.TFRecordDataset(tfrecord_files).map(parse
                                                         ) # , num_parallel_reads=tf.data.AUTOTUNE
    # dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE) 
    return dataset


def bandwise_normalize(input_tensor, epsilon=1e-8):
    input_tensor = tf.cast(input_tensor, tf.float32)
    min_val = tf.reduce_min(input_tensor, axis=2, keepdims=True)
    max_val = tf.reduce_max(input_tensor, axis=2, keepdims=True)
    denom = max_val - min_val
    denom = tf.where(tf.abs(denom) < epsilon, epsilon, denom)
    normalized_tensor = (input_tensor - min_val) / denom
    return normalized_tensor



def pad_to_multiple(image, TILE_HT, TILE_WD):
    print(f"Pad2Multiple's Starting Shape: {tf.shape(image)}")
    # shape = tf.shape(image)
    # height, width = shape[0], shape[1]
    # target_height = tf.cast(tf.math.ceil(height / TILE_HT) * TILE_HT, tf.int32)
    # target_width = tf.cast(tf.math.ceil(width / TILE_WD) * TILE_WD, tf.int32)
    # padded_image = tf.image.resize_with_crop_or_pad(image, target_height, target_width)
    
    # Obtain the dynamic shape of the image
    shape = tf.shape(image)
    height, width = shape[0], shape[1]
    
    # Calculate the padding needed to make the height and width a multiple of TILE_HT and TILE_WD
    pad_height = tf.math.floormod(-height, TILE_HT)
    pad_width = tf.math.floormod(-width, TILE_WD)
    
    # Calculate padding for top/bottom and left/right
    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left
    
    # Create the paddings tensor
    paddings = tf.constant([[pad_top, pad_bottom], [pad_left, pad_right], [0, 0]])
    
    # Pad the image
    padded_image = tf.pad(image, paddings, mode="CONSTANT", constant_values=0)
    
    print(f"Pad2Multiple's padded Shape: {tf.shape(padded_image)}")
    return padded_image



def tile_image(fullimg, CHANNELS, TILE_HT, TILE_WD):
    fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
    images = tf.expand_dims(fullimg, axis=0)
    tiles = tf.image.extract_patches(
        images=images,
        sizes=[1, TILE_HT, TILE_WD, 1],
        strides=[1, TILE_HT, TILE_WD, 1],
        rates=[1, 1, 1, 1],
        padding="VALID",
    )

    tiles = tf.squeeze(tiles, axis=0)
    tiles = tf.cast(tiles, tf.float32)
    tiles = tf.reshape(tiles, [tf.shape(tiles)[0], tf.shape(tiles)[1], TILE_HT, TILE_WD, CHANNELS])
    return tiles


def sampling(label_image, threshold_percentage=99.9):
    num_zeros = tf.reduce_sum(
        tf.cast(tf.equal(label_image, 0), tf.float32), axis=[2, 3, 4]
    )
    total_elements = tf.cast(tf.reduce_prod(tf.shape(label_image)[2:]), tf.float32)
    percentage_zeros = (num_zeros / total_elements) * 100.0
    boolean_mask = percentage_zeros <= threshold_percentage
    sampled_tensor = tf.cast(percentage_zeros >= threshold_percentage, tf.int32)
    return boolean_mask, sampled_tensor



def one_hot_encoding(label_tensor):
    float_labels = tf.squeeze(label_tensor, axis=-1)
    num_classes = tf.cast(tf.reduce_max(float_labels) + 1, tf.int32)
    one_hot_encoded_images = tf.one_hot(tf.dtypes.cast(float_labels, tf.int32), depth=num_classes)
    return one_hot_encoded_images



def parsing(
    dataset,
    patch_height,
    patch_width,
    threshold_percentage,
    image_channels,
    label_channels,
):
    image_patch_tensors_list = []
    label_patch_tensors_list = []
    
    
    def _parse(parsed_example):
        image_shape = tf.sparse.to_dense(parsed_example["image_shape"])
        image = tf.reshape(tf.sparse.to_dense(parsed_example["image"]), image_shape)
        label_shape = tf.sparse.to_dense(parsed_example["label_shape"])
        label = tf.reshape(tf.sparse.to_dense(parsed_example["label"]), label_shape)
        
        # Assuming image_channels is known and fixed
        # image.set_shape([None, None, image_channels])
        # label.set_shape([None, None, 1])
        
        # image normalization
        image = bandwise_normalize(image)

        # image and label patching
        image_patches = tile_image(image, image_channels, patch_height, patch_width)
        label_patches = tile_image(label, label_channels, patch_height, patch_width)

        # sampling
        sampled_mask, sampled_tensor = sampling(label_patches, threshold_percentage)
        sampled_image_patches = tf.boolean_mask(image_patches, sampled_mask)
        sampled_label_patches = tf.boolean_mask(label_patches, sampled_mask)

        # one-hot encoding
        sampled_label_patches = one_hot_encoding(sampled_label_patches)

        return sampled_image_patches, sampled_label_patches
    
    parsed_dataset = dataset.map(lambda x: _parse(x))
    for sampled_image_patches, sampled_label_patches in parsed_dataset:
        image_patch_tensors_list.append(sampled_image_patches)
        label_patch_tensors_list.append(sampled_label_patches)

    return image_patch_tensors_list, label_patch_tensors_list


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client(project="gislogics")
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print("File {} uploaded to {}.".format(source_file_name, destination_blob_name))

    
def train_test_datasets(
    input_directory,
    patch_height,
    patch_width,
    image_channels,
    label_channels,
    threshold_percentage,
    batch_size,
    image_names
):
    dataset = create_dataset(input_directory, image_names)
    image_patch_tensors_list, label_patch_tensors_list = parsing(
        dataset=dataset,
        patch_height=patch_height,
        patch_width=patch_width,
        image_channels=image_channels,
        label_channels=label_channels,
        threshold_percentage=threshold_percentage,
    )
    
    combined_images = tf.concat(image_patch_tensors_list, axis=0)
    combined_labels = tf.concat(label_patch_tensors_list, axis=0)

    # Shuffle the combined data
    combined_dataset = tf.data.Dataset.from_tensor_slices(
        (combined_images, combined_labels)
    )
    combined_dataset = combined_dataset.shuffle(buffer_size=combined_images.shape[0])

    # Split the combined dataset into training and validation sets
    train_size = int(0.8 * combined_images.shape[0])
    train_dataset = combined_dataset.take(train_size)
    val_dataset = combined_dataset.skip(train_size)

    # Batch the data using TensorFlow's Dataset API
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)

    return train_dataset, val_dataset
    

def load_config(file_path):
    with open(file_path, 'r') as f:
        config = json.load(f)
    return config








# Modeling--------------------------------------------------
config = load_config('./pipeline_scripts/config.json')
from google.cloud import storage
from tensorflow.keras.models import save_model, load_model
from tensorflow.python.lib.io import file_io
# Convert img_size to a tuple
config["img_size"] = tuple(config["img_size"])

# Now you can access the config values normally
model_path = config.get("model_path")
model_name = config.get("model_name")
test_image_path = config.get("test_image_path")
img_size = config.get("img_size")
num_bands = config.get("num_bands")
num_classes = config.get("num_classes")
gcs_path = config.get("google_storage_path")
gcs_tfrecords = config.get("gcs_tfrecords")


# Load the model if a checkpoint exists
# Load the model if a checkpoint exists
import gcsfs

fs = gcsfs.GCSFileSystem(project='gislogics')  # Set up GCS connection

if tf.io.gfile.exists(model_path):  # Accurately check for file existence in GCS
    # Load existing model
    model = load_model(model_path)  # Handle GCS authentication here if needed
    print("Loaded model from checkpoint")
else:
    # If no checkpoint exists, create a new model
    model = get_model(
        img_size=img_size, num_classes=num_classes, num_bands=num_bands
    )
    model.compile(optimizer="adam", loss="categorical_crossentropy")
    print("Created a new model")

    # # Save the new model to the model_path in GCS
    save_model(model, model_path)
    print("Saved new model to:", model_path)

    
    
    
    
# Continue training
from pipeline_scripts.eval import *
from pipeline_scripts.prediction import *

def train(**kwargs):
    input_directory = kwargs.get("input_directory")
    threshold_percentage = kwargs.get("threshold_percentage")
    image_channels = kwargs.get("image_channels")
    label_channels = kwargs.get("label_channels")
    patch_height = kwargs.get("patch_height")
    patch_width = kwargs.get("patch_width")
    batch_size = kwargs.get("batch_size")
    num_classes = kwargs.get("num_classes")  
    bucket_name = kwargs.get("bucket_name")

    image_names = kwargs.get("image_names")
    # img_size = (patch_height, patch_width)
    
    

    # get the train and test datasets
    train_dataset, val_dataset = train_test_datasets(
        gcs_tfrecords,
        patch_height,
        patch_width,
        image_channels,
        label_channels,
        threshold_percentage,
        batch_size,
        image_names
    )
    print("Train and Valid datasets are created")

    # create img_size
    model = get_model(
        img_size= img_size, 
        num_classes=num_classes, 
        num_bands=image_channels
    )
    
    
    # compilation of model, with custom metric
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy", 
        metrics=[np.mean(compute_metrics(new_ground_truth, new_predict)[0])]
    )

    
    # Early stopping after 5 epochs 
    early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,  
    restore_best_weights=True,  
    verbose=1  
    )
    
    # Agroforestry Class
    Agroforestry = 18
    
    # including custom metrics in callbacks
    custom_metrics_callback = keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs: logs.update({
        "class_wise_iou": compute_metrics(new_ground_truth, new_predict)[0][Agroforestry],
        "class_wise_dice_score": compute_metrics(new_ground_truth, new_predict)[1][Agroforestry],
        "class_wise_accuracy": compute_metrics(new_ground_truth, new_predict)[2][Agroforestry],
        "class_wise_precision": compute_metrics(new_ground_truth, new_predict)[3][Agroforestry],
        "class_wise_recall": compute_metrics(new_ground_truth, new_predict)[4][Agroforestry],
        "mean_iou": compute_metrics(new_ground_truth, new_predict)[5],
        "min_class_wise_iou": np.min(metrics_results[0]),
        "max_class_wise_iou": np.max(metrics_results[0]),
        "epoch": epoch,
        "loss": logs["loss"],
        "val_loss": logs["val_loss"]}))

    
    # callbacks and logging
    csv_logger = keras.callbacks.CSVLogger(
    input_directory + "logs/" + f"training_logs_{model_name}.csv",
    append=True
    )

    custom_metrics_csv_logger = CustomMetricsCSVLogger(
        input_params["input_directory"] + "logs/" + f"training_logs_{model_name}.csv",
        append=True
    )
    
    
    log_dir = "gs://tf_records_bucket/tf_records/Untitled Folder/logs/"  # Specify the directory to save logs
    tensorboard_callback = TensorBoard(log_dir=log_dir)

    output_of_image = "gs://tf_records_bucket/tf_records/Untitled Folder/output"
    
    # Combine all callbacks
    all_callbacks = [
        keras.callbacks.ModelCheckpoint(model_path + model_name, save_best_only=False),
        tensorboard_callback,
        custom_metrics_callback,
        custom_metrics_csv_logger,  # Add the custom_metrics_csv_logger here
        # early_stopping,
        PredictSegmentationCallback(test_image_path, output_of_image)
    ]

    model_history = model.fit(
        train_dataset,
        epochs=3,
        callbacks=all_callbacks,
        batch_size=32,
        validation_data=val_dataset,
    )

    if early_stopping.stopped_epoch > 0:
        print(f"Training stopped at epoch {early_stopping.stopped_epoch} due to early stopping.")
    else:
        print("Training completed all epochs.")
    # Save the model after training
    # model.save(model_path + model_name)
    # print("Model saved locally")

    upload_blob(gcs_path)
    print("Uploaded to cloud storage successfully")



if __name__ == "__main__":
    input_params = {
        "input_directory": "gs://tf_records_bucket/tf_records/Untitled Folder/",  # make sure / is there at the end,
        "bucket_name": "tf_records_bucket",
        "threshold_percentage": 99.9,
        "image_names" : [],
        "image_channels": 8,  # 8 bands images as input
        "label_channels": 1,
        "patch_height": 8,
        "patch_width": 8,
        "batch_size": 32,
        "num_classes": 23,
        "model_path": "trained_model/",
    }  # makesure there is a slash at the end of the path  # Choose an appropriate batch size

    train(**input_params)



  Using cached rasterio-1.3.9-cp310-cp310-manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl.metadata (3.4 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)
Using cached rasterio-1.3.9-cp310-cp310-manylinux2014_x86_64.whl (20.6 MB)
Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
Using cached affine-2.4.0-py3-none-any.whl (15 kB)
Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached geopandas-0.14.3-py3-none-any.whl.metadata (1.5 kB)
  Using cached fiona-1.9.6-cp310-cp310-manylinux2014_x86_64.whl.metadata (50 kB)
  Using cached pyproj-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
Using cached geopandas-0.14.3-py3-none-any.whl (1.1 MB)
Using cached fiona-1.9.6-cp310-cp310-manylinux2014_x86

2024-03-09 22:59:54.464931: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 23:00:08.672702: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load li

Loaded model from checkpoint
prediction.clip_tiff, tiff gs://tf_records_bucket/labels/area2_0530_2022_8bands.tif
(3694, 4560, 8)
(3694, 4560, 8)
dims (3696, 4560, 8)
129/129 [==============================] - 11s 76ms/step
Time taken for predictions: 12.892645597457886 seconds
segmentation_patches_reshaped.shape (462, 570, 8, 8)
row_indices_patch (462,)
col_indices_patch (570,)
stitched_array (3696, 4560)
Time taken including stitching: 14.933786630630493 seconds
prediction.clip_tiff, tiff gs://tf_records_bucket/labels/continuous_label_raster.tif
<unknown>
<unknown>


ValueError: in user code:

    File "/tmp/ipykernel_1/3823001887.py", line 420, in None  *
        lambda x: _parse(x)
    File "/tmp/ipykernel_1/3823001887.py", line 407, in _parse  *
        image_patches = tile_image(image, image_channels, patch_height, patch_width)
    File "/home/jupyter/eval_and_pred/pipeline_scripts/prediction.py", line 135, in tile_image  *
        fullimg = pad_to_multiple(fullimg, TILE_HT, TILE_WD)
    File "/home/jupyter/eval_and_pred/pipeline_scripts/prediction.py", line 117, in pad_to_multiple  *
        height, width, channels = image.shape

    ValueError: Cannot iterate over a shape with unknown rank.
